In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Se van a revisar los datasets correspondientes a movies para poder unificarlos

In [ ]:
import pandas as pd

# Rutas de los archivos
movies_path = '/content/drive/MyDrive/TFM/Datasets/movies.csv'
ratings_path = '/content/drive/MyDrive/TFM/Datasets/ratings.csv'
tags_path = '/content/drive/MyDrive/TFM/Datasets/tags.csv'
links_path = '/content/drive/MyDrive/TFM/Datasets/links.csv'

# Cargar los datasets
movies_df = pd.read_csv(movies_path)
ratings_df = pd.read_csv(ratings_path)
tags_df = pd.read_csv(tags_path)
links_df = pd.read_csv(links_path)

# Revisar la estructura de cada dataset
print("Movies Dataset Structure:")
print(movies_df.info(), "\n")
print(movies_df.head())

print("\nRatings Dataset Structure:")
print(ratings_df.info(), "\n")
print(ratings_df.head())

print("\nTags Dataset Structure:")
print(tags_df.info(), "\n")
print(tags_df.head())

print("\nLinks Dataset Structure:")
print(links_df.info(), "\n")
print(links_df.head())


Movies Dataset Structure:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB
None 

   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  

Ratings Dataset Structure:
<

 usar movieId como clave principal para unir los datos de los cuatro datasets, ya que está presente en cada uno de ellos, y así tener el dataset de movies

In [ ]:
import pandas as pd

# Cargar los datasets
movies_df = pd.read_csv('/content/drive/MyDrive/TFM/Datasets/movies.csv')
ratings_df = pd.read_csv('/content/drive/MyDrive/TFM/Datasets/ratings.csv')
tags_df = pd.read_csv('/content/drive/MyDrive/TFM/Datasets/tags.csv')
links_df = pd.read_csv('/content/drive/MyDrive/TFM/Datasets/links.csv')

# Unir datasets en una estructura base usando movieId
# 1. Unir movies con ratings
merged_df = pd.merge(movies_df, ratings_df, on='movieId', how='left')

# 2. Unir el resultado con tags
merged_df = pd.merge(merged_df, tags_df, on=['movieId', 'userId'], how='left')

# 3. Unir el resultado con links
merged_df = pd.merge(merged_df, links_df, on='movieId', how='left')

# Visualizar estructura del dataset unificado
print("Estructura del dataset combinado:")



Estructura del dataset combinado:


In [ ]:
merged_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102695 entries, 0 to 102694
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   movieId      102695 non-null  int64  
 1   title        102695 non-null  object 
 2   genres       102695 non-null  object 
 3   userId       102677 non-null  float64
 4   rating       102677 non-null  float64
 5   timestamp_x  102677 non-null  float64
 6   tag          3476 non-null    object 
 7   timestamp_y  3476 non-null    float64
 8   imdbId       102695 non-null  int64  
 9   tmdbId       102682 non-null  float64
dtypes: float64(5), int64(2), object(3)
memory usage: 7.8+ MB


El data set queda muy desequilibrado dado q las filas de tag son solo 3476, se decide dejar un dataset solo con los registrios que contengan tag para el modelo BERT y otro data set grande sin tag, vamos con el grande

In [ ]:
import pandas as pd

# Cargar los datasets
movies_df = pd.read_csv('/content/drive/MyDrive/TFM/Datasets/movies.csv')
ratings_df = pd.read_csv('/content/drive/MyDrive/TFM/Datasets/ratings.csv')
links_df = pd.read_csv('/content/drive/MyDrive/TFM/Datasets/links.csv')

# 1. Filtrar las películas con al menos 5 calificaciones para asegurar representatividad
popular_movies = ratings_df.groupby('movieId').filter(lambda x: len(x) >= 5)

# 2. Unir datasets de películas populares y links, usando solo los registros comunes
merged_df = movies_df.merge(popular_movies, on='movieId', how='inner') \
                     .merge(links_df, on='movieId', how='inner')

# Visualizar la estructura del dataset equilibrado
print("Estructura del dataset equilibrado para recomendaciones:")
print(merged_df.info())
print(merged_df.head())


Estructura del dataset equilibrado para recomendaciones:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90274 entries, 0 to 90273
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   movieId    90274 non-null  int64  
 1   title      90274 non-null  object 
 2   genres     90274 non-null  object 
 3   userId     90274 non-null  int64  
 4   rating     90274 non-null  float64
 5   timestamp  90274 non-null  int64  
 6   imdbId     90274 non-null  int64  
 7   tmdbId     90274 non-null  float64
dtypes: float64(2), int64(4), object(2)
memory usage: 5.5+ MB
None
   movieId             title                                       genres  \
0        1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy   
1        1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy   
2        1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy   
3        1  Toy Story (1995)  Adventure|Animation|Children|C

In [ ]:
merged_df.head()

,movieId,title,genres,userId,rating,timestamp,imdbId,tmdbId
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703,114709,862.0
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962,114709,862.0
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946,114709,862.0
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970,114709,862.0
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483,114709,862.0


In [ ]:
merged_df.movieId.unique()

array([     1,      2,      3, ..., 183897, 187593, 187595])

In [ ]:
merged_df.movieId.value_counts()

,count
movieId,
356,329
318,317
296,307
593,279
2571,278
...,...
33903,5
190,5
34334,5


Analizando columna por columna para tener un mejor entendimiento se decide que para el titulo se va a separar el nombre de la película que queda en la columna tittle y el año pasará a otra columna year para tener mas interacción

In [ ]:
 merged_df.title.unique()

array(['Toy Story (1995)', 'Jumanji (1995)', 'Grumpier Old Men (1995)',
       ..., 'Isle of Dogs (2018)', 'Deadpool 2 (2018)',
       'Solo: A Star Wars Story (2018)'], dtype=object)

In [ ]:
merged_df.title.value_counts()

,count
title,
Forrest Gump (1994),329
"Shawshank Redemption, The (1994)",317
Pulp Fiction (1994),307
"Silence of the Lambs, The (1991)",279
"Matrix, The (1999)",278
...,...
"Edukators, The (Die Fetten Jahre sind vorbei) (2004)",5
Safe (1995),5
Stealth (2005),5


In [ ]:
merged_df.head()

,movieId,title,genres,userId,rating,timestamp_x,tag,timestamp_y,imdbId,tmdbId
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1.0,4.0,9.649827e+08,NaN,NaN,114709,862.0
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5.0,4.0,8.474350e+08,NaN,NaN,114709,862.0
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7.0,4.5,1.106636e+09,NaN,NaN,114709,862.0
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15.0,2.5,1.510578e+09,NaN,NaN,114709,862.0
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17.0,4.5,1.305696e+09,NaN,NaN,114709,862.0


In [ ]:
import re

# Extraer el año del título
merged_df['year'] = merged_df['title'].apply(lambda x: int(re.search(r'\((\d{4})\)', x).group(1)) if re.search(r'\((\d{4})\)', x) else None)

# Eliminar el año del título, manteniendo solo el nombre de la película
merged_df['title'] = merged_df['title'].apply(lambda x: re.sub(r'\s*\(\d{4}\)', '', x))

# Revisar la estructura después de los cambios
print(merged_df[['title', 'year']].head())


       title  year
0  Toy Story  1995
1  Toy Story  1995
2  Toy Story  1995
3  Toy Story  1995
4  Toy Story  1995


In [ ]:
merged_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90274 entries, 0 to 90273
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   movieId    90274 non-null  int64  
 1   title      90274 non-null  object 
 2   genres     90274 non-null  object 
 3   userId     90274 non-null  int64  
 4   rating     90274 non-null  float64
 5   timestamp  90274 non-null  int64  
 6   imdbId     90274 non-null  int64  
 7   tmdbId     90274 non-null  float64
 8   year       90274 non-null  int64  
dtypes: float64(2), int64(5), object(2)
memory usage: 6.2+ MB


In [ ]:
merged_df.year.unique()

array([1995, 1994, 1996, 1976, 1992, 1967, 1977, 1993, 1965, 1982, 1990,
       1991, 1989, 1937, 1940, 1969, 1981, 1970, 1968, 1988, 1964, 1972,
       1952, 1951, 1957, 1961, 1958, 1954, 1934, 1944, 1959, 1960, 1963,
       1942, 1941, 1953, 1939, 1950, 1946, 1945, 1955, 1938, 1947, 1935,
       1956, 1975, 1974, 1971, 1973, 1979, 1987, 1986, 1980, 1978, 1985,
       1966, 1962, 1983, 1949, 1984, 1948, 1933, 1931, 1922, 1997, 1998,
       1930, 1927, 1943, 1999, 2000, 1926, 1921, 1936, 1925, 2001, 2002,
       2003, 1928, 1920, 2004, 1932, 2005, 2006, 1902, 2007, 2008, 2009,
       2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018])

In [ ]:
merged_df.genres.value_counts()

,count
genres,
Comedy,6145
Drama,4966
Comedy|Romance,3535
Comedy|Drama|Romance,2713
Drama|Romance,2427
...,...
Comedy|Drama|Horror,5
Comedy|Drama|Romance|Thriller,5
Action|Crime|Mystery,5


In [ ]:
merged_df.genres.unique()

array(['Adventure|Animation|Children|Comedy|Fantasy',
       'Adventure|Children|Fantasy', 'Comedy|Romance',
       'Comedy|Drama|Romance', 'Comedy', 'Action|Crime|Thriller',
       'Adventure|Children', 'Action', 'Action|Adventure|Thriller',
       'Comedy|Horror', 'Adventure|Animation|Children', 'Drama',
       'Action|Adventure|Romance', 'Crime|Drama', 'Drama|Romance',
       'Action|Comedy|Crime|Drama|Thriller', 'Comedy|Crime|Thriller',
       'Crime|Drama|Horror|Mystery|Thriller', 'Drama|Sci-Fi',
       'Children|Drama', 'Adventure|Drama|Fantasy|Mystery|Sci-Fi',
       'Mystery|Sci-Fi|Thriller', 'Drama|War', 'Action|Crime|Drama',
       'Action|Adventure|Fantasy', 'Comedy|Drama|Thriller',
       'Mystery|Thriller', 'Animation|Children|Drama|Musical|Romance',
       'Crime|Mystery|Thriller', 'Drama|Thriller', 'Comedy|Crime',
       'Action|Sci-Fi|Thriller', 'Action|Comedy|Horror|Thriller',
       'Comedy|Drama', 'Crime|Drama|Romance', 'Action|Adventure|Drama',
       'Children|Come

Limitar la combinación de géneros a un máximo de tres palabras.
Reorganizar o mapear combinaciones largas en las tres más representativas según la primera, segunda, y tercera palabras del género, respetando el orden en el que aparecen.

In [ ]:
# Ordenar alfabéticamente cada combinación de géneros para evitar duplicados por orden de palabras
merged_df['genres'] = merged_df['genres'].apply(lambda x: '|'.join(sorted(x.split('|'))))

# Verificar las combinaciones únicas después de la ordenación
print(merged_df['genres'].value_counts())




genres
Comedy                      6145
Drama                       4966
Action|Adventure|Sci-Fi     4755
Comedy|Romance              3535
Comedy|Drama|Romance        3135
                            ... 
Children|Fantasy|Musical       5
Adventure|Thriller             5
Fantasy|Western                5
Action|Horror|Mystery          5
War                            5
Name: count, Length: 324, dtype: int64


In [ ]:
merged_df.genres.unique()

array(['Adventure|Animation|Children', 'Adventure|Children|Fantasy',
       'Comedy|Romance', 'Comedy|Drama|Romance', 'Comedy',
       'Action|Crime|Thriller', 'Adventure|Children', 'Action',
       'Action|Adventure|Thriller', 'Comedy|Horror', 'Drama',
       'Action|Adventure|Romance', 'Crime|Drama', 'Drama|Romance',
       'Action|Comedy|Crime', 'Comedy|Crime|Thriller',
       'Crime|Drama|Horror', 'Drama|Sci-Fi', 'Children|Drama',
       'Adventure|Drama|Fantasy', 'Mystery|Sci-Fi|Thriller', 'Drama|War',
       'Action|Crime|Drama', 'Action|Adventure|Fantasy',
       'Comedy|Drama|Thriller', 'Mystery|Thriller',
       'Animation|Children|Drama', 'Crime|Mystery|Thriller',
       'Drama|Thriller', 'Comedy|Crime', 'Action|Sci-Fi|Thriller',
       'Action|Comedy|Horror', 'Comedy|Drama', 'Crime|Drama|Romance',
       'Action|Adventure|Drama', 'Children|Comedy', 'Action|Thriller',
       'Drama|Horror|Thriller', 'Comedy|Horror|Romance',
       'Adventure|Comedy|Crime', 'Adventure|Children

Enfoque de Agrupación en Géneros Generales


Acción (Action, Adventure, Thriller)

Drama (Drama, Romance, War, Western)

Ciencia Ficción/Fantasía (Sci-Fi, Fantasy)

Animación (Animation, Children, Musical, Family)

Documental (Documentary, IMAX)

Comedia (Comedy, Romance)

Terror (Horror, Mystery, Crime, Film-Noir)

In [ ]:
# Actualizar el mapeo de géneros según la guía en inglés
def map_genres(genre_str):
    genre_mapping = {
        'Action': 'Action',
        'Adventure': 'Action',
        'Thriller': 'Suspense',
        'Sci-Fi': 'Science Fiction',
        'Fantasy': 'Fantasy',
        'Animation': 'Family',
        'Children': 'Children',
        'Family': 'Family',
        'Comedy': 'Comedy',
        'Drama': 'Drama',
        'Romance': 'Romance',
        'War': 'Drama',
        'Western': 'Action',
        'Musical': 'Musical',
        'Documentary': 'Documentary',
        'IMAX': 'Documentary',
        'Horror': 'Horror',
        'Mystery': 'Suspense',
        'Crime': 'Suspense',
        'Film-Noir': 'Suspense'
    }

    # Aplicar el mapeo y reemplazar según las categorías definidas
    genres = [genre_mapping.get(genre, genre) for genre in genre_str.split('|')]
    # Ordenar y eliminar duplicados en cada combinación
    return '|'.join(sorted(set(genres)))

# Aplicar el mapeo en la columna 'genres'
merged_df['genres'] = merged_df['genres'].apply(map_genres)

# Verificar la nueva distribución de géneros después de la categorización
print(merged_df['genres'].value_counts())


genres
Drama|Suspense              6257
Comedy                      6145
Drama                       5889
Action|Science Fiction      5525
Action|Suspense             5299
                            ... 
Documentary|Drama|Family       5
Family|Fantasy|Musical         5
Family|Horror|Suspense         5
Children                       5
Children|Fantasy|Musical       5
Name: count, Length: 166, dtype: int64


In [ ]:
# Limitar los géneros a un máximo de dos combinaciones y ordenar alfabéticamente para evitar duplicados por orden de palabras
merged_df['genres'] = merged_df['genres'].apply(lambda x: '|'.join(sorted(x.split('|')[:2])))

# Verificar la distribución de géneros después de reducir a combinaciones de dos palabras
print(merged_df['genres'].value_counts())


genres
Action|Drama                8046
Comedy|Drama                7878
Action|Comedy               7747
Action|Science Fiction      7214
Comedy                      6652
Drama|Suspense              6257
Drama                       6172
Action|Suspense             5299
Action|Children             4523
Comedy|Romance              3659
Drama|Romance               3451
Suspense                    2283
Action|Fantasy              2034
Children|Comedy             1757
Comedy|Suspense             1647
Action                      1455
Horror|Suspense             1274
Drama|Science Fiction       1039
Children|Drama               858
Action|Horror                858
Action|Family                775
Drama|Horror                 769
Comedy|Horror                735
Action|Romance               728
Comedy|Fantasy               707
Drama|Fantasy                691
Horror|Science Fiction       669
Children|Family              632
Documentary                  617
Action|Documentary           544
Com

In [ ]:
# Eliminar filas donde el género es '(no genres listed)'
merged_df = merged_df[merged_df['genres'] != '(no genres listed)']

# Verificar la distribución de géneros después de la eliminación
print(merged_df['genres'].value_counts())


genres
Action|Drama                8046
Comedy|Drama                7878
Action|Comedy               7747
Action|Science Fiction      7214
Comedy                      6652
Drama|Suspense              6257
Drama                       6172
Action|Suspense             5299
Action|Children             4523
Comedy|Romance              3659
Drama|Romance               3451
Suspense                    2283
Action|Fantasy              2034
Children|Comedy             1757
Comedy|Suspense             1647
Action                      1455
Horror|Suspense             1274
Drama|Science Fiction       1039
Action|Horror                858
Children|Drama               858
Action|Family                775
Drama|Horror                 769
Comedy|Horror                735
Action|Romance               728
Comedy|Fantasy               707
Drama|Fantasy                691
Horror|Science Fiction       669
Children|Family              632
Documentary                  617
Action|Documentary           544
Com

In [ ]:
merged_df.genres.unique()

array(['Action|Children', 'Comedy|Romance', 'Comedy|Drama', 'Comedy',
       'Action|Suspense', 'Action', 'Comedy|Horror', 'Drama',
       'Action|Romance', 'Drama|Suspense', 'Drama|Romance',
       'Action|Comedy', 'Comedy|Suspense', 'Drama|Horror',
       'Drama|Science Fiction', 'Children|Drama', 'Action|Drama',
       'Science Fiction|Suspense', 'Action|Fantasy', 'Suspense',
       'Action|Science Fiction', 'Children|Comedy', 'Documentary',
       'Action|Documentary', 'Horror', 'Fantasy|Horror',
       'Horror|Science Fiction', 'Drama|Musical',
       'Comedy|Science Fiction', 'Drama|Fantasy', 'Horror|Suspense',
       'Action|Horror', 'Children', 'Children|Family', 'Action|Family',
       'Family', 'Comedy|Fantasy', 'Comedy|Musical', 'Musical', 'Romance',
       'Fantasy', 'Science Fiction', 'Comedy|Family'], dtype=object)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90267 entries, 0 to 90266
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   movieId    90267 non-null  int64  
 1   title      90267 non-null  object 
 2   genres     90267 non-null  object 
 3   userId     90267 non-null  int64  
 4   rating     90267 non-null  float64
 5   timestamp  90267 non-null  int64  
 6   imdbId     90267 non-null  int64  
 7   tmdbId     90267 non-null  float64
 8   year       90267 non-null  int64  
dtypes: float64(2), int64(5), object(2)
memory usage: 6.2+ MB


In [ ]:
df.userId.unique()

array([  1,   5,   7,  15,  17,  18,  19,  21,  27,  31,  32,  33,  40,
        43,  44,  45,  46,  50,  54,  57,  63,  64,  66,  68,  71,  73,
        76,  78,  82,  86,  89,  90,  91,  93,  96,  98, 103, 107, 112,
       119, 121, 124, 130, 132, 134, 135, 137, 140, 141, 144, 145, 151,
       153, 155, 156, 159, 160, 161, 166, 167, 169, 171, 177, 178, 179,
       182, 185, 186, 191, 193, 200, 201, 202, 206, 213, 214, 216, 217,
       219, 220, 223, 226, 229, 232, 233, 234, 239, 240, 247, 249, 252,
       254, 263, 264, 266, 269, 270, 273, 274, 275, 276, 277, 279, 280,
       282, 283, 288, 290, 291, 292, 293, 298, 304, 307, 314, 322, 323,
       328, 330, 332, 334, 336, 337, 339, 341, 347, 350, 353, 357, 359,
       364, 367, 372, 373, 378, 380, 381, 382, 385, 389, 391, 396, 399,
       401, 411, 412, 414, 420, 422, 432, 434, 436, 438, 443, 448, 451,
       453, 456, 460, 462, 468, 469, 470, 471, 474, 476, 477, 480, 483,
       484, 488, 490, 492, 500, 504, 509, 514, 517, 522, 524, 52

In [ ]:
df.userId.value_counts()

,count
userId,
414,2132
599,1756
474,1390
448,1285
274,1177
...,...
320,18
544,18
578,17


In [ ]:
df.rating.unique()

array([4. , 4.5, 2.5, 3.5, 3. , 5. , 0.5, 2. , 1.5, 1. ])

In [ ]:
df.rating.value_counts()

,count
rating,
4.0,24547
3.0,17999
5.0,12450
3.5,11524
4.5,7765
2.0,6419
2.5,4702
1.0,2362
1.5,1389


In [ ]:
df.timestamp.unique()

array([ 964982703,  847434962, 1106635946, ..., 1527977920, 1537674946,
       1529899556])

In [ ]:
df.timestamp.value_counts()

,count
timestamp,
1459787998,120
1459787997,118
1459787996,77
828124616,35
829760897,29
...,...
1362016835,1
1459367469,1
1496205180,1


La columna timestamp suele almacenar la fecha y hora en que un usuario hizo una calificación para una película. Es decir, cada entrada representa el momento específico en el que se registró el puntaje (valor en la columna rating) dado por el usuario (userId) a una película (movieId).

Este dato puede ser útil para:

Análisis de patrones de comportamiento de usuario: Observar cuándo un usuario es más activo en calificar contenido.
Evolución temporal de preferencias: Detectar cambios en los gustos o intereses de los usuarios a lo largo del tiempo.
Recomendaciones basadas en recencia: Priorizar contenido calificado o visto recientemente para entender mejor las preferencias actuales.

In [ ]:
# Convertir la columna timestamp a un formato de fecha y hora legible
from datetime import datetime

df['timestamp'] = df['timestamp'].apply(lambda x: datetime.fromtimestamp(x))

# Verificar los primeros valores después de la conversión
print(df['timestamp'].head())


0   2000-07-30 18:45:03
1   1996-11-08 06:36:02
2   2005-01-25 06:52:26
3   2017-11-13 12:59:30
4   2011-05-18 05:28:03
Name: timestamp, dtype: datetime64[ns]


In [ ]:
df.timestamp.unique()

<DatetimeArray>
['2000-07-30 18:45:03', '1996-11-08 06:36:02', '2005-01-25 06:52:26',
 '2017-11-13 12:59:30', '2011-05-18 05:28:03', '2016-02-11 16:56:56',
 '2000-08-08 03:33:57', '2014-08-09 21:14:38', '2000-07-04 04:34:22',
 '1996-12-13 08:43:36',
 ...
 '2018-09-18 01:49:54', '2018-06-28 01:14:25', '2018-07-11 23:19:53',
 '2018-09-17 02:18:46', '2018-06-25 03:59:39', '2018-06-14 00:02:30',
 '2018-09-13 21:05:28', '2018-06-02 22:18:40', '2018-09-23 03:55:46',
 '2018-06-25 04:05:56']
Length: 75794, dtype: datetime64[ns]

In [ ]:
 df.to_csv('/content/drive/MyDrive/TFM/Datasets/integracionv1.csv', index=False)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/TFM/Datasets/integracionv1.csv')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90267 entries, 0 to 90266
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   movieId    90267 non-null  int64  
 1   title      90267 non-null  object 
 2   genres     90267 non-null  object 
 3   userId     90267 non-null  int64  
 4   rating     90267 non-null  float64
 5   timestamp  90267 non-null  object 
 6   imdbId     90267 non-null  int64  
 7   tmdbId     90267 non-null  float64
 8   year       90267 non-null  int64  
dtypes: float64(2), int64(4), object(3)
memory usage: 6.2+ MB


In [ ]:
df.genres.unique()

array(['Action|Children', 'Comedy|Romance', 'Comedy|Drama', 'Comedy',
       'Action|Suspense', 'Action', 'Comedy|Horror', 'Drama',
       'Action|Romance', 'Drama|Suspense', 'Drama|Romance',
       'Action|Comedy', 'Comedy|Suspense', 'Drama|Horror',
       'Drama|Science Fiction', 'Children|Drama', 'Action|Drama',
       'Science Fiction|Suspense', 'Action|Fantasy', 'Suspense',
       'Action|Science Fiction', 'Children|Comedy', 'Documentary',
       'Action|Documentary', 'Horror', 'Fantasy|Horror',
       'Horror|Science Fiction', 'Drama|Musical',
       'Comedy|Science Fiction', 'Drama|Fantasy', 'Horror|Suspense',
       'Action|Horror', 'Children', 'Children|Family', 'Action|Family',
       'Family', 'Comedy|Fantasy', 'Comedy|Musical', 'Musical', 'Romance',
       'Fantasy', 'Science Fiction', 'Comedy|Family'], dtype=object)

In [ ]:
df.timestamp.unique()

array(['2000-07-30 18:45:03', '1996-11-08 06:36:02',
       '2005-01-25 06:52:26', ..., '2018-06-02 22:18:40',
       '2018-09-23 03:55:46', '2018-06-25 04:05:56'], dtype=object)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90267 entries, 0 to 90266
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   movieId    90267 non-null  int64  
 1   title      90267 non-null  object 
 2   genres     90267 non-null  object 
 3   userId     90267 non-null  int64  
 4   rating     90267 non-null  float64
 5   timestamp  90267 non-null  object 
 6   imdbId     90267 non-null  int64  
 7   tmdbId     90267 non-null  float64
 8   year       90267 non-null  int64  
dtypes: float64(2), int64(4), object(3)
memory usage: 6.2+ MB


In [ ]:
df.tmdbId.unique()

array([   862.,   8844.,  15602., ..., 399174., 383498., 348350.])

In [ ]:
df.tmdbId.value_counts()

,count
tmdbId,
13.0,329
278.0,317
680.0,307
274.0,279
603.0,278
...,...
11092.0,5
11509.0,5
483.0,5


Bien ahora vamos a generar el otro data set con las tag para BERT

In [ ]:
import pandas as pd

# Cargar el dataset de tags
tags_path = '/content/drive/MyDrive/TFM/Datasets/tags.csv'
tags_df = pd.read_csv(tags_path)

# Crear un nuevo DataFrame solo con las columnas relevantes para el análisis de texto
bert_tags_df = tags_df[['movieId', 'userId', 'tag']].copy()

# Añadir una columna vacía donde se almacenarán las emociones predichas por BERT
bert_tags_df['emotion'] = None  # Esto será llenado después del procesamiento con BERT

# Guardar el nuevo dataset para análisis posterior
bert_tags_path = '/content/drive/MyDrive/TFM/Datasets/bert_tags.csv'
bert_tags_df.to_csv(bert_tags_path, index=False)

print("Nuevo dataset para BERT creado y guardado en:", bert_tags_path)
print(bert_tags_df.head())


Nuevo dataset para BERT creado y guardado en: /content/drive/MyDrive/TFM/Datasets/bert_tags.csv
   movieId  userId              tag emotion
0    60756       2            funny    None
1    60756       2  Highly quotable    None
2    60756       2     will ferrell    None
3    89774       2     Boxing story    None
4    89774       2              MMA    None


In [ ]:
bert_tags_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3683 entries, 0 to 3682
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  3683 non-null   int64 
 1   userId   3683 non-null   int64 
 2   tag      3683 non-null   object
 3   emotion  0 non-null      object
dtypes: int64(2), object(2)
memory usage: 115.2+ KB


In [ ]:
# Cargar los datasets necesarios
main_df = pd.read_csv('/content/drive/MyDrive/TFM/Datasets/integracionv1.csv')  # Este es el dataset principal ya procesado
bert_tags_df = pd.read_csv('/content/drive/MyDrive/TFM/Datasets/bert_tags.csv')  # Dataset de etiquetas preparado para BERT

# Realizar la combinación entre ambos datasets para encontrar coincidencias basadas en el movieId
merged_df = pd.merge(bert_tags_df, main_df, on='movieId', how='inner', suffixes=('_tag', '_main'))

# Mostrar los resultados
total_coincidencias = merged_df.shape[0]
unique_movieId_coincidencias = merged_df['movieId'].nunique()

print(f"Total de coincidencias en registros: {total_coincidencias}")
print(f"Total de 'movieId' únicos con coincidencias: {unique_movieId_coincidencias}")
print("Muestra de registros coincidentes:")
print(merged_df.head())


Total de coincidencias en registros: 231828
Total de 'movieId' únicos con coincidencias: 1126
Muestra de registros coincidentes:
   movieId  userId_tag    tag  emotion          title  genres  userId_main  \
0    60756           2  funny      NaN  Step Brothers  Comedy            2   
1    60756           2  funny      NaN  Step Brothers  Comedy           18   
2    60756           2  funny      NaN  Step Brothers  Comedy           62   
3    60756           2  funny      NaN  Step Brothers  Comedy           68   
4    60756           2  funny      NaN  Step Brothers  Comedy           73   

   rating            timestamp  imdbId   tmdbId  year  
0     5.0  2015-10-24 19:29:40  838283  12133.0  2008  
1     3.0  2016-02-17 22:50:49  838283  12133.0  2008  
2     3.5  2018-06-13 23:59:36  838283  12133.0  2008  
3     2.5  2010-03-20 22:14:03  838283  12133.0  2008  
4     4.5  2016-05-25 17:10:21  838283  12133.0  2008  


In [ ]:
bert_tags_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3683 entries, 0 to 3682
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  3683 non-null   int64  
 1   userId   3683 non-null   int64  
 2   tag      3683 non-null   object 
 3   emotion  0 non-null      float64
dtypes: float64(1), int64(2), object(1)
memory usage: 115.2+ KB


In [ ]:
bert_tags_df.tag.unique()

array(['funny', 'Highly quotable', 'will ferrell', ..., 'gun fu',
       'heroic bloodshed', 'Heroic Bloodshed'], dtype=object)

In [ ]:
# Asegurarse de que el nombre del dataset principal sea correcto (df en este caso)
filtered_bert_tags_df = bert_tags_df[bert_tags_df['movieId'].isin(df['movieId'].unique())]

# Verificar la estructura y mostrar una muestra del resultado
print("Filtered bert_tags_df Structure:")
print(filtered_bert_tags_df.info(), "\n")
print(filtered_bert_tags_df.head())



Filtered bert_tags_df Structure:
<class 'pandas.core.frame.DataFrame'>
Index: 3033 entries, 0 to 3682
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  3033 non-null   int64  
 1   userId   3033 non-null   int64  
 2   tag      3033 non-null   object 
 3   emotion  0 non-null      float64
dtypes: float64(1), int64(2), object(1)
memory usage: 118.5+ KB
None 

   movieId  userId              tag  emotion
0    60756       2            funny      NaN
1    60756       2  Highly quotable      NaN
2    60756       2     will ferrell      NaN
3    89774       2     Boxing story      NaN
4    89774       2              MMA      NaN


In [ ]:
# Filtrar bert_tags_df para que solo incluya los movieId presentes en df
consistent_movie_ids = df['movieId'].unique()  # Obtener los movieId únicos del dataset principal
filtered_bert_tags_df = bert_tags_df[bert_tags_df['movieId'].isin(consistent_movie_ids)]  # Filtrar por consistencia

# Verificar la estructura y mostrar una muestra del resultado
print("Filtered bert_tags_df Structure (Consistent):")
print(f"{filtered_bert_tags_df['movieId'].nunique()} unique movieId values")  # Confirmación del número único de movieId
print(filtered_bert_tags_df.info())  # Información general sobre el dataset filtrado
print(filtered_bert_tags_df.head())  # Muestra de los primeros registros



Filtered bert_tags_df Structure (Consistent):
1126 unique movieId values
<class 'pandas.core.frame.DataFrame'>
Index: 3033 entries, 0 to 3682
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  3033 non-null   int64  
 1   userId   3033 non-null   int64  
 2   tag      3033 non-null   object 
 3   emotion  0 non-null      float64
dtypes: float64(1), int64(2), object(1)
memory usage: 118.5+ KB
None
   movieId  userId              tag  emotion
0    60756       2            funny      NaN
1    60756       2  Highly quotable      NaN
2    60756       2     will ferrell      NaN
3    89774       2     Boxing story      NaN
4    89774       2              MMA      NaN


In [ ]:
filtered_bert_tags_df.to_csv('/content/drive/MyDrive/TFM/Datasets/data_tags_BERT.csv', index=False)

In [ ]:
bert = pd.read_csv('/content/drive/MyDrive/TFM/Datasets/data_tags_BERT.csv')

In [ ]:
bert.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3033 entries, 0 to 3032
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  3033 non-null   int64  
 1   userId   3033 non-null   int64  
 2   tag      3033 non-null   object 
 3   emotion  0 non-null      float64
dtypes: float64(1), int64(2), object(1)
memory usage: 94.9+ KB


Ahora se integrará el data set de movies con el de música

1. Exploración Inicial y Preparación del Dataset

Cargamos y revisamos la estructura original del dataset de música. Identificamos las columnas principales: Artist, Album, Track, y la falta de una columna de Genre adecuada para muchos registros.

Observamos que el género era Unknown para la mayoría de las entradas.

2. Asignación Manual de Géneros
Creamos un mapeo inicial manual (artist_genre_map) para asignar géneros a los artistas más comunes y conocidos, con el fin de cubrir una porción significativa del dataset.
Aplicamos este mapeo a los registros de los artistas presentes, reduciendo considerablemente la cantidad de géneros Unknown.

3. Agrupación por Álbum para Inferir Géneros
Para artistas y álbumes específicos conocidos por un género en particular, asignamos el género de forma coherente a todos los tracks de ese álbum. Esta agrupación mejoró la precisión al reducir ambigüedades en álbumes con varios tracks.

4. Balanceo y Refinamiento
Verificamos la distribución de los géneros asignados. Algunos géneros, como Rock, estaban sobrerrepresentados.
Aplicamos una estrategia de balanceo que limitó el tamaño de Rock a 20,000 muestras para equilibrarlo con otros géneros.
Esto ayudó a evitar el sesgo hacia el género Rock, asegurando una representación más equitativa de otros géneros.

5. Predicción de Géneros Faltantes Usando un Modelo de Clasificación
Entrenamos un modelo de clasificación de texto (basado en Artist, Album, y Track) para predecir los géneros de las entradas restantes con género Unknown.
Evaluamos la precisión del modelo mediante validación cruzada y aseguramos que alcanzara un nivel de precisión alto antes de aplicarlo al dataset.
Asignamos géneros predichos a los registros Unknown y verificamos la coherencia.

6. Verificación Final y Ajustes
Revisamos muestras al azar para confirmar la coherencia entre el artista y el género asignado.
Hicimos los ajustes necesarios en casos específicos donde la predicción podría desviarse del género real del artista.
Guardamos el dataset final con todos los géneros asignados y balanceados, ahora listo para su integración con el dataset de películas.

7. Exportación Final
Guardamos el dataset tratado como musica_integracionv2, asegurando una estructura y consistencia de géneros adecuada para su uso en el modelo de recomendación basado en emociones.
Este tratamiento nos permite integrar el dataset de música al conjunto de datos de películas de manera precisa, aportando una base sólida para los análisis y predicciones del sistema de recomendación basado en emociones que queremos desarrollar con BERT y Deep Learning.

In [ ]:
import pandas as pd

# Cargar los archivos
movies_path = '/content/drive/MyDrive/TFM/Datasets/integracionv1.csv'
music_path = '/content/drive/MyDrive/TFM/Datasets/Last.fm_data.csv'

# Cargar los datasets
movies_df = pd.read_csv(movies_path)
music_df = pd.read_csv(music_path)

# Explorar la estructura y las primeras filas de cada dataset
print("Movies Dataset Structure:")
print(movies_df.info(), "\n")
print(movies_df.head())

print("\nMusic Dataset Structure:")
print(music_df.info(), "\n")
print(music_df.head())

# Explorar columnas únicas para identificar posibles llaves de unión y contenido
print("\nUnique Columns in Movies Dataset:")
print(movies_df.nunique())

print("\nUnique Columns in Music Dataset:")
print(music_df.nunique())


Movies Dataset Structure:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90267 entries, 0 to 90266
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   movieId    90267 non-null  int64  
 1   title      90267 non-null  object 
 2   genres     90267 non-null  object 
 3   userId     90267 non-null  int64  
 4   rating     90267 non-null  float64
 5   timestamp  90267 non-null  object 
 6   imdbId     90267 non-null  int64  
 7   tmdbId     90267 non-null  float64
 8   year       90267 non-null  int64  
dtypes: float64(2), int64(4), object(3)
memory usage: 6.2+ MB
None 

   movieId      title           genres  userId  rating            timestamp  \
0        1  Toy Story  Action|Children       1     4.0  2000-07-30 18:45:03   
1        1  Toy Story  Action|Children       5     4.0  1996-11-08 06:36:02   
2        1  Toy Story  Action|Children       7     4.5  2005-01-25 06:52:26   
3        1  Toy Story  Action|Children   

In [ ]:
# Limpiar el dataset de música eliminando la columna Unnamed: 0
music_df = music_df.drop(columns=['Unnamed: 0'])

# Ver coincidencias potenciales entre Username y userId
unique_userIds = movies_df['userId'].unique()
unique_usernames = music_df['Username'].unique()

# Mostrar coincidencias entre userId en movies_df y Username en music_df (si existen)
matching_users = [user for user in unique_usernames if user in unique_userIds]
print("Usuarios coincidentes entre datasets:", matching_users)


Usuarios coincidentes entre datasets: []


In [ ]:
movies_df.genres.unique()

array(['Action|Children', 'Comedy|Romance', 'Comedy|Drama', 'Comedy',
       'Action|Suspense', 'Action', 'Comedy|Horror', 'Drama',
       'Action|Romance', 'Drama|Suspense', 'Drama|Romance',
       'Action|Comedy', 'Comedy|Suspense', 'Drama|Horror',
       'Drama|Science Fiction', 'Children|Drama', 'Action|Drama',
       'Science Fiction|Suspense', 'Action|Fantasy', 'Suspense',
       'Action|Science Fiction', 'Children|Comedy', 'Documentary',
       'Action|Documentary', 'Horror', 'Fantasy|Horror',
       'Horror|Science Fiction', 'Drama|Musical',
       'Comedy|Science Fiction', 'Drama|Fantasy', 'Horror|Suspense',
       'Action|Horror', 'Children', 'Children|Family', 'Action|Family',
       'Family', 'Comedy|Fantasy', 'Comedy|Musical', 'Musical', 'Romance',
       'Fantasy', 'Science Fiction', 'Comedy|Family'], dtype=object)

In [ ]:
# Definir el mapeo de géneros entre películas y música
genre_mapping = {
    'Action': 'Rock',
    'Children': 'Children’s Music',
    'Comedy': 'Pop',
    'Suspense': 'Techno/Industrial',
    'Drama': 'Blues/R&B',
    'Horror': 'Metal',
    'Fantasy': 'Alternative',
    'Romance': 'Ballad/Romantic',
    'Science Fiction': 'Electronic/Dance',
    'Documentary': 'Classical/Jazz',
    'Family': 'Children’s Music',
    'Musical': 'Soundtrack/Broadway',
    # Agregar combinaciones específicas
    'Action|Children': 'Rock/Children’s Music',
    'Comedy|Romance': 'Pop/Ballad',
    'Comedy|Drama': 'Pop/Blues',
    'Comedy|Horror': 'Pop/Metal',
    'Drama|Suspense': 'Blues/Techno',
    'Action|Suspense': 'Rock/Techno',
    'Action|Romance': 'Rock/Ballad',
    'Drama|Romance': 'Blues/Ballad',
    'Comedy|Suspense': 'Pop/Techno',
    'Drama|Horror': 'Blues/Metal',
    'Drama|Science Fiction': 'Blues/Electronic',
    'Children|Drama': 'Children’s Music/Blues',
    'Action|Drama': 'Rock/Blues',
    'Science Fiction|Suspense': 'Electronic/Techno',
    'Action|Fantasy': 'Rock/Alternative',
    'Action|Science Fiction': 'Rock/Electronic',
    'Children|Comedy': 'Children’s Music/Pop',
    'Action|Documentary': 'Rock/Classical',
    'Fantasy|Horror': 'Alternative/Metal',
    'Horror|Science Fiction': 'Metal/Electronic',
    'Drama|Musical': 'Blues/Soundtrack',
    'Comedy|Science Fiction': 'Pop/Electronic',
    'Drama|Fantasy': 'Blues/Alternative',
    'Horror|Suspense': 'Metal/Techno',
    'Action|Horror': 'Rock/Metal',
    'Children|Family': 'Children’s Music',
    'Action|Family': 'Rock/Children’s Music',
    'Comedy|Fantasy': 'Pop/Alternative',
    'Comedy|Musical': 'Pop/Soundtrack',
    'Fantasy|Romance': 'Alternative/Ballad',
    'Science Fiction|Romance': 'Electronic/Ballad',
    'Comedy|Family': 'Pop/Children’s Music',
}

# Aplicar el mapeo al dataset de películas y de música
movies_df['mapped_genre'] = movies_df['genres'].map(genre_mapping)
music_df['mapped_genre'] = music_df['Genre'].map(genre_mapping)

# Mostrar una muestra de cada dataset para verificar
print("Movies dataset with mapped genres:")
print(movies_df[['title', 'genres', 'mapped_genre']].head())
print("\nMusic dataset with mapped genres:")
print(music_df[['Artist', 'Track', 'Genre', 'mapped_genre']].head())


KeyError: 'Genre'

In [ ]:
music_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 166153 entries, 0 to 166152
Data columns (total 8 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  166153 non-null  int64 
 1   Username    166153 non-null  object
 2   Artist      166153 non-null  object
 3   Track       166153 non-null  object
 4   Album       166141 non-null  object
 5   Date        166153 non-null  object
 6   Time        166153 non-null  object
 7   Genre       166153 non-null  object
dtypes: int64(1), object(7)
memory usage: 10.1+ MB


In [ ]:
music_df.Username.unique()

array(['Babs_05', 'franhale', 'eartle', 'massdosage', 'Knapster01',
       'jonocole', 'isaac', 'lobsterclaw', 'jajo', 'mremond', 'Orlenay'],
      dtype=object)

In [ ]:
music_df.Artist.unique()

array(['Isobel Campbell', 'The Coral', 'Gidge', ..., 'edwynn page',
       'Roy Montgomery', 'DJ Spooky'], dtype=object)

In [ ]:
music_df.Track.unique()

array(['The Circus Is Leaving Town', 'Dusty Wreath',
       'Honey Child What Can I Do?', ..., 'Pinnacles of You',
       'Arrows of Time', 'Closest approach to your Orbit'], dtype=object)

In [ ]:
music_df.Album.unique()

array(['Ballad of the Broken Seas', 'Lockdown Sessions', 'Perimeter', ...,
       'Peng! (2018 Remaster)', 'Spotify Session', 'Fucking Money Man'],
      dtype=object)

In [ ]:
music_df.Date.unique()

array(['31 Jan 2021', '30 Jan 2021', '29 Jan 2021', '26 Jan 2021',
       '25 Jan 2021', '19 Jan 2021', '28 Jan 2021', '27 Jan 2021',
       '22 Jan 2021', '23 Jan 2021', '10 Jan 2021', '24 Jan 2021',
       '20 Jan 2021', '18 Jan 2021', '15 Jan 2021', '14 Jan 2021',
       '04 Jan 2021', '01 Jan 2021', '21 Jan 2021', '06 Jan 2021',
       '12 Jan 2021', '11 Jan 2021', '17 Jan 2021', '09 Jan 2021',
       '16 Jan 2021', '13 Jan 2021', '08 Jan 2021', '05 Jan 2021',
       '02 Jan 2021', '03 Jan 2021', '07 Jan 2021'], dtype=object)

In [ ]:
music_df.Time.unique()

array([' 23:36', ' 23:32', ' 23:28', ..., ' 07:02', ' 07:47', ' 06:44'],
      dtype=object)

In [ ]:
# Cargar el dataset de música
music_df = pd.read_csv('/content/drive/MyDrive/TFM/Datasets/Last.fm_data.csv')

# Contar la frecuencia de cada artista
top_artists = music_df['Artist'].value_counts().head(60)  # Podemos ajustar el número según lo necesitemos

# Mostrar los artistas más comunes
print("Top 50 Artistas más comunes:")
print(top_artists)


Top 50 Artistas más comunes:
Artist
Sophie                      1181
Madlib                      1074
Bicep                        766
Taylor Swift                 649
Arlo Parks                   593
Morrissey                    498
Depeche Mode                 469
Coldplay                     393
New Order                    367
MF DOOM                      366
James Blake                  352
Radiohead                    348
Phoebe Bridgers              345
Metallica                    344
Lana Del Rey                 338
Metric                       331
Kendrick Lamar               327
ABBA                         325
Björk                        322
Lady Gaga                    313
Bill Wurtz                   307
The Beatles                  307
Steve Earle & The Dukes      300
Johann Sebastian Bach        295
Charli XCX                   294
The Weeknd                   294
Eminem                       287
Dua Lipa                     286
Rhye                         285
Weezer 

In [ ]:
music_df.Artist.value_counts()

,count
Artist,
Sophie,1181
Madlib,1074
Bicep,766
Taylor Swift,649
Arlo Parks,593
...,...
Talos,1
Vybz Kartel,1
LadiPOE,1


In [ ]:
unique_artists = music_df['Artist'].unique()

# Mostrar la cantidad total de artistas y los primeros 50 como muestra
len(unique_artists), unique_artists[100]

(22823, 'Ralph Vaughan Williams')

In [ ]:
import pandas as pd

# Cargar el dataset de música actualizado
music_df = pd.read_csv('/content/drive/MyDrive/TFM/Datasets/Last.fm_data.csv')

# Crear un diccionario de mapeo para asignar géneros a los artistas más comunes
artist_genre_map = {
    'Taylor Swift': 'Pop', 'Lana Del Rey': 'Pop', 'Lady Gaga': 'Pop', 'The Weeknd': 'Pop',
    'Britney Spears': 'Pop', 'Ariana Grande': 'Pop', 'Dua Lipa': 'Pop', 'ABBA': 'Pop', 'Charli XCX': 'Pop',
    'Coldplay': 'Rock', 'Radiohead': 'Rock', 'Foo Fighters': 'Rock', 'The Beatles': 'Rock',
    'Linkin Park': 'Rock', 'Red Hot Chili Peppers': 'Rock', 'Pink Floyd': 'Rock', 'The Cure': 'Rock', 'The Smiths': 'Rock',
    'Kendrick Lamar': 'Hip-Hop/Rap', 'MF DOOM': 'Hip-Hop/Rap', 'Eminem': 'Hip-Hop/Rap',
    'Sophie': 'Electronic', 'Bicep': 'Electronic', 'Daft Punk': 'Electronic', 'Grimes': 'Electronic',
    'The Avalanches': 'Electronic', 'Autechre': 'Electronic',
    'Phoebe Bridgers': 'Alternative/Indie', 'Tame Impala': 'Alternative/Indie',
    'Weezer': 'Alternative/Indie', 'Arctic Monkeys': 'Alternative/Indie',
    'Johann Sebastian Bach': 'Classical', 'Ólafur Arnalds': 'Classical',
    'Metallica': 'Metal', 'Sabaton': 'Metal', 'Tool': 'Metal',
    'John Lee Hooker': 'Blues/Jazz',
    'Steve Earle & The Dukes': 'Country/Folk',
    'Boards of Canada': 'Experimental', 'Loscil': 'Experimental'
}

# Asignar géneros al dataset de música usando el diccionario
music_df['Genre'] = music_df['Artist'].map(artist_genre_map)

# Rellenar NaN en la columna de género con "Unknown"
music_df['Genre'].fillna('Unknown', inplace=True)

# Verificar los cambios y la distribución de géneros
print(music_df['Genre'].value_counts())
print(music_df.head())

# Guardar el dataset actualizado si es necesario
music_df.to_csv('/content/drive/MyDrive/TFM/Datasets/Last.fm_data_with_genres.csv', index=False)


<ipython-input-29-1edf92569d31>:28: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  music_df['Genre'].fillna('Unknown', inplace=True)


Genre
Unknown              153691
Pop                    2900
Electronic             2879
Rock                   2373
Alternative/Indie      1046
Hip-Hop/Rap             980
Metal                   752
Classical               518
Experimental            467
Country/Folk            300
Blues/Jazz              247
Name: count, dtype: int64
   Unnamed: 0 Username           Artist                          Track  \
0           0  Babs_05  Isobel Campbell     The Circus Is Leaving Town   
1           1  Babs_05  Isobel Campbell                   Dusty Wreath   
2           2  Babs_05  Isobel Campbell     Honey Child What Can I Do?   
3           3  Babs_05  Isobel Campbell  It's Hard To Kill A Bad Thing   
4           4  Babs_05  Isobel Campbell                Saturday's Gone   

                       Album         Date    Time    Genre  
0  Ballad of the Broken Seas  31 Jan 2021   23:36  Unknown  
1  Ballad of the Broken Seas  31 Jan 2021   23:32  Unknown  
2  Ballad of the Broken Seas  31

In [ ]:
# Diccionario expandido para asignar géneros a más artistas populares
additional_artist_genre_map = {
    'Sophie': 'Electronic',
    'Madlib': 'Hip-Hop/Rap',
    'Bicep': 'Electronic',
    'Taylor Swift': 'Pop',
    'Arlo Parks': 'Alternative/Indie',
    'Morrissey': 'Rock',
    'Depeche Mode': 'Rock',
    'Coldplay': 'Alternative/Indie',
    'New Order': 'Electronic',
    'MF DOOM': 'Hip-Hop/Rap',
    'James Blake': 'Electronic',
    'Radiohead': 'Alternative/Indie',
    'Phoebe Bridgers': 'Alternative/Indie',
    'Metallica': 'Metal',
    'Lana Del Rey': 'Pop',
    'Metric': 'Alternative/Indie',
    'Kendrick Lamar': 'Hip-Hop/Rap',
    'ABBA': 'Pop',
    'Björk': 'Electronic',
    'Lady Gaga': 'Pop',
    'The Beatles': 'Rock',
    'Steve Earle & The Dukes': 'Country/Folk',
    'Johann Sebastian Bach': 'Classical',
    'Charli XCX': 'Pop',
    'The Weeknd': 'Pop',
    'Eminem': 'Hip-Hop/Rap',
    'Dua Lipa': 'Pop',
    'Rhye': 'Electronic',
    'Weezer': 'Rock',
    'Daft Punk': 'Electronic',
    'Ocean Waves For Sleep': 'Experimental',
    'Tame Impala': 'Alternative/Indie',
    'Khruangbin': 'Alternative/Indie',
    'Beyoncé': 'Pop',
    'John Lee Hooker': 'Blues/Jazz',
    'Boards of Canada': 'Electronic',
    'Red Hot Chili Peppers': 'Rock',
    'Pink Floyd': 'Rock',
    'Rage Against the Machine': 'Rock',
    'Dorian Electra': 'Pop',
    'The Cure': 'Rock',
    'Foo Fighters': 'Rock',
    'The Smiths': 'Rock',
    'Linkin Park': 'Rock',
    'PJ Harvey': 'Alternative/Indie',
    'Britney Spears': 'Pop',
    'Autechre': 'Electronic',
    'Ariana Grande': 'Pop',
    'Joni Mitchell': 'Country/Folk',
    'Tool': 'Metal'
}

# Aplicar el mapeo expandido al dataset de música
music_df['Genre'] = music_df['Artist'].map(additional_artist_genre_map).fillna(music_df['Genre'])

# Verificar los resultados después del mapeo expandido
print("Distribución de géneros después del mapeo adicional:")
print(music_df['Genre'].value_counts())
print(music_df.head())


Distribución de géneros después del mapeo adicional:
Genre
Unknown              147751
Electronic             4448
Pop                    3384
Rock                   3108
Alternative/Indie      2906
Hip-Hop/Rap            2054
Metal                   752
Classical               518
Country/Folk            495
Experimental            490
Blues/Jazz              247
Name: count, dtype: int64
   Unnamed: 0 Username           Artist                          Track  \
0           0  Babs_05  Isobel Campbell     The Circus Is Leaving Town   
1           1  Babs_05  Isobel Campbell                   Dusty Wreath   
2           2  Babs_05  Isobel Campbell     Honey Child What Can I Do?   
3           3  Babs_05  Isobel Campbell  It's Hard To Kill A Bad Thing   
4           4  Babs_05  Isobel Campbell                Saturday's Gone   

                       Album         Date    Time    Genre  
0  Ballad of the Broken Seas  31 Jan 2021   23:36  Unknown  
1  Ballad of the Broken Seas  31 Jan 202

In [ ]:
import pandas as pd

# Cargar el dataset de música
music_df = pd.read_csv('/content/drive/MyDrive/TFM/Datasets/Last.fm_data.csv')

# Verificar si la columna 'Genre' existe; si no, crearla con valor 'Unknown'
if 'Genre' not in music_df.columns:
    music_df['Genre'] = 'Unknown'

# Diccionario de géneros específicos por artista
artist_genre_map = {
    'Sophie': 'Electronic', 'Madlib': 'Hip-Hop/Rap', 'Bicep': 'Electronic', 'Taylor Swift': 'Pop',
    'Arlo Parks': 'Alternative/Indie', 'Morrissey': 'Rock', 'Depeche Mode': 'Rock',
    'Coldplay': 'Alternative/Indie', 'New Order': 'Electronic', 'MF DOOM': 'Hip-Hop/Rap',
    'James Blake': 'Electronic', 'Radiohead': 'Alternative/Indie', 'Phoebe Bridgers': 'Alternative/Indie',
    'Metallica': 'Metal', 'Lana Del Rey': 'Pop', 'Metric': 'Alternative/Indie', 'Kendrick Lamar': 'Hip-Hop/Rap',
    'ABBA': 'Pop', 'Björk': 'Electronic', 'Lady Gaga': 'Pop', 'The Beatles': 'Rock',
    'Steve Earle & The Dukes': 'Country/Folk', 'Johann Sebastian Bach': 'Classical', 'Charli XCX': 'Pop',
    'The Weeknd': 'Pop', 'Eminem': 'Hip-Hop/Rap', 'Dua Lipa': 'Pop', 'Rhye': 'Electronic',
    'Weezer': 'Rock', 'Daft Punk': 'Electronic', 'Ocean Waves For Sleep': 'Experimental', 'Tame Impala': 'Alternative/Indie',
    'Khruangbin': 'Alternative/Indie', 'Beyoncé': 'Pop', 'John Lee Hooker': 'Blues/Jazz', 'Boards of Canada': 'Electronic',
    'Red Hot Chili Peppers': 'Rock', 'Pink Floyd': 'Rock', 'Rage Against the Machine': 'Rock', 'Dorian Electra': 'Pop',
    'The Cure': 'Rock', 'Foo Fighters': 'Rock', 'The Smiths': 'Rock', 'Linkin Park': 'Rock', 'PJ Harvey': 'Alternative/Indie',
    'Britney Spears': 'Pop', 'Autechre': 'Electronic', 'Ariana Grande': 'Pop', 'Joni Mitchell': 'Country/Folk', 'Tool': 'Metal'
}

# Aplicar el mapeo de géneros en la columna 'Artist'
music_df['Genre'] = music_df['Artist'].map(artist_genre_map).fillna(music_df['Genre'])

# Rellenar NaN en 'Album' con un valor predeterminado
music_df['Album'].fillna('Unknown Album', inplace=True)

# Agrupar por 'Album' y asignar el género más frecuente en cada álbum
album_genres = music_df.groupby('Album')['Genre'].apply(lambda x: x.mode().iloc[0] if not x.mode().empty else 'Unknown')
music_df['Genre'] = music_df.apply(lambda row: album_genres[row['Album']] if row['Genre'] == 'Unknown' else row['Genre'], axis=1)

# Ver la distribución de géneros después de la agrupación por álbum
print("\nDistribución de géneros después de la agrupación por álbum:")
print(music_df['Genre'].value_counts())

# Mostrar una muestra de los primeros registros para verificar la asignación de géneros
print("\nMuestra de los primeros registros con género asignado:")
print(music_df.head())


<ipython-input-34-b691bbc81cd5>:31: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  music_df['Album'].fillna('Unknown Album', inplace=True)



Distribución de géneros después de la agrupación por álbum:
Genre
Unknown              148810
Electronic             4091
Pop                    3439
Rock                   3128
Alternative/Indie      2760
Hip-Hop/Rap            2071
Metal                   535
Country/Folk            505
Classical               300
Experimental            266
Blues/Jazz              248
Name: count, dtype: int64

Muestra de los primeros registros con género asignado:
   Unnamed: 0 Username           Artist                          Track  \
0           0  Babs_05  Isobel Campbell     The Circus Is Leaving Town   
1           1  Babs_05  Isobel Campbell                   Dusty Wreath   
2           2  Babs_05  Isobel Campbell     Honey Child What Can I Do?   
3           3  Babs_05  Isobel Campbell  It's Hard To Kill A Bad Thing   
4           4  Babs_05  Isobel Campbell                Saturday's Gone   

                       Album         Date    Time    Genre  
0  Ballad of the Broken Seas  31 Jan 

In [ ]:
# 1. Revisar y contar álbumes comunes
common_albums = music_df['Album'].value_counts().head(20)  # Top 20 álbumes para inspección
print("Álbumes más comunes:")
print(common_albums)

# Manualmente podemos inspeccionar algunos de estos álbumes y asignarles géneros si son consistentes.

# 2. Ampliar el diccionario `artist_genre_map`
# Añade más artistas aquí según la popularidad o el conocimiento del género de estos artistas
artist_genre_map.update({
    'Sufjan Stevens': 'Alternative/Indie',
    'Nine Inch Nails': 'Rock',
    'Fleetwood Mac': 'Rock',
    'Beethoven': 'Classical',
    'Ella Fitzgerald': 'Blues/Jazz',
    # Continúa añadiendo artistas relevantes para este proyecto
})

# Aplicar nuevamente el mapeo con el diccionario actualizado
music_df['Genre'] = music_df['Artist'].map(artist_genre_map)

# Completar cualquier valor desconocido restante con "Unknown"
music_df['Genre'].fillna('Unknown', inplace=True)

# Revisar la distribución de géneros después del refinamiento
print("\nDistribución de géneros después del refinamiento:")
print(music_df['Genre'].value_counts())


Álbumes más comunes:
Album
OIL OF EVERY PEARL'S UN-INSIDES                        696
Sound Ancestors                                        669
Isles                                                  438
The HMV / Parlophone Singles 1988-1995                 396
Collapsed in Sunbeams                                  325
Cascades (Dirt Road Version)                           297
Greatest Hits                                          267
Waves of the Sea, Ocean and Lakes                      266
Hamilton (Original Broadway Cast Recording)            250
Madilb Medicine Show #11 - Low Budget High Fi Music    250
Bicep                                                  239
Home                                                   228
XCOM Legacy (Original Soundtrack)                      217
evermore                                               214
Punisher                                               208
Vespertine                                             197
J.T.                         

<ipython-input-35-6d8093e33bab>:23: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  music_df['Genre'].fillna('Unknown', inplace=True)


In [ ]:
# Diccionario ampliado de géneros por álbum para los álbumes más comunes
album_genre_map = {
    'OIL OF EVERY PEARL\'S UN-INSIDES': 'Electronic',
    'Sound Ancestors': 'Hip-Hop/Rap',
    'Isles': 'Electronic',
    'The HMV / Parlophone Singles 1988-1995': 'Rock',
    'Collapsed in Sunbeams': 'Alternative/Indie',
    'Cascades (Dirt Road Version)': 'Country/Folk',
    'Greatest Hits': 'Pop',  # Este puede variar según el artista, ajustar si es posible
    'Waves of the Sea, Ocean and Lakes': 'Experimental',
    'Hamilton (Original Broadway Cast Recording)': 'Musical',
    'Madilb Medicine Show #11 - Low Budget High Fi Music': 'Hip-Hop/Rap',
    'Bicep': 'Electronic',
    'Home': 'Pop',
    'XCOM Legacy (Original Soundtrack)': 'Soundtrack',
    'evermore': 'Pop',
    'Punisher': 'Alternative/Indie',
    'Vespertine': 'Electronic',
    'J.T.': 'Country/Folk',
    'Purple Mountains': 'Alternative/Indie',
    'OK Human': 'Rock',
    'Madlib Medicine Show': 'Hip-Hop/Rap',
    # Ampliar con más álbumes conocidos si se desea
}

# Asignar el género según el álbum usando el diccionario ampliado
music_df['Genre'] = music_df.apply(
    lambda row: album_genre_map[row['Album']] if row['Album'] in album_genre_map else row['Genre'],
    axis=1
)

# Llenar valores desconocidos como 'Unknown'
music_df['Genre'].fillna('Unknown', inplace=True)

# Verificar la distribución y una muestra de los datos
print("Distribución de géneros después del refinamiento adicional:")
print(music_df['Genre'].value_counts())

# Muestra de los primeros registros con el género asignado
print("\nMuestra de los primeros registros con género asignado:")
print(music_df.head())


Distribución de géneros después del refinamiento adicional:
Genre
Unknown              147631
Pop                    3880
Electronic             3788
Rock                   3344
Alternative/Indie      2777
Hip-Hop/Rap            2064
Country/Folk            792
Metal                   533
Blues/Jazz              316
Classical               295
Experimental            266
Musical                 250
Soundtrack              217
Name: count, dtype: int64

Muestra de los primeros registros con género asignado:
   Unnamed: 0 Username           Artist                          Track  \
0           0  Babs_05  Isobel Campbell     The Circus Is Leaving Town   
1           1  Babs_05  Isobel Campbell                   Dusty Wreath   
2           2  Babs_05  Isobel Campbell     Honey Child What Can I Do?   
3           3  Babs_05  Isobel Campbell  It's Hard To Kill A Bad Thing   
4           4  Babs_05  Isobel Campbell                Saturday's Gone   

                       Album         Date  

<ipython-input-36-5b384e0022a2>:33: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  music_df['Genre'].fillna('Unknown', inplace=True)


In [ ]:
# Ampliar el mapeo de álbumes a géneros con más álbumes comunes
album_genre_map.update({
    'evermore': 'Pop',
    'The HMV / Parlophone Singles 1988-1995': 'Rock',
    'Greatest Hits': 'Rock',
    'Hamilton (Original Broadway Cast Recording)': 'Musical',
    'Purple Mountains': 'Alternative/Indie',
    'OK Human': 'Alternative/Indie',
    # Añadir más álbumes conocidos aquí según sea necesario
})

# Actualizar los géneros en el dataset de música con el mapeo de álbumes refinado
music_df['Genre'] = music_df.apply(lambda x: album_genre_map.get(x['Album'], x['Genre']), axis=1)

# Mapeo adicional por artista para cubrir más casos
artist_genre_map.update({
    'Isobel Campbell': 'Alternative/Indie',
    'Johann Sebastian Bach': 'Classical',
    'Morrissey': 'Rock',
    # Añadir más artistas conocidos aquí según sea necesario
})

# Asignar género basado en el artista si el género aún es 'Unknown'
music_df['Genre'] = music_df.apply(lambda x: artist_genre_map.get(x['Artist'], x['Genre']) if x['Genre'] == 'Unknown' else x['Genre'], axis=1)

# Verificar la distribución de géneros después del refinamiento adicional
print("Distribución de géneros después del refinamiento adicional:")
print(music_df['Genre'].value_counts())

# Mostrar una muestra de los primeros registros con el nuevo mapeo aplicado
print("\nMuestra de los primeros registros con género asignado:")
print(music_df.head())


Distribución de géneros después del refinamiento adicional:
Genre
Unknown              147494
Electronic             3788
Pop                    3613
Rock                   3440
Alternative/Indie      3085
Hip-Hop/Rap            2064
Country/Folk            792
Metal                   533
Blues/Jazz              316
Classical               295
Experimental            266
Musical                 250
Soundtrack              217
Name: count, dtype: int64

Muestra de los primeros registros con género asignado:
   Unnamed: 0 Username           Artist                          Track  \
0           0  Babs_05  Isobel Campbell     The Circus Is Leaving Town   
1           1  Babs_05  Isobel Campbell                   Dusty Wreath   
2           2  Babs_05  Isobel Campbell     Honey Child What Can I Do?   
3           3  Babs_05  Isobel Campbell  It's Hard To Kill A Bad Thing   
4           4  Babs_05  Isobel Campbell                Saturday's Gone   

                       Album         Date  

In [ ]:
# Guardar el dataset actualizado si es necesario
music_df.to_csv('/content/drive/MyDrive/TFM/Datasets/Last.fm_data_with_genres.csv', index=False)

In [ ]:
music_df = pd.read_csv('/content/drive/MyDrive/TFM/Datasets/Last.fm_data_with_genres.csv')

In [ ]:
music_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 166153 entries, 0 to 166152
Data columns (total 8 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  166153 non-null  int64 
 1   Username    166153 non-null  object
 2   Artist      166153 non-null  object
 3   Track       166153 non-null  object
 4   Album       166153 non-null  object
 5   Date        166153 non-null  object
 6   Time        166153 non-null  object
 7   Genre       166153 non-null  object
dtypes: int64(1), object(7)
memory usage: 10.1+ MB


In [ ]:
music_df.Genre.unique()

array(['Alternative/Indie', 'Unknown', 'Classical', 'Electronic',
       'Hip-Hop/Rap', 'Pop', 'Rock', 'Metal', 'Blues/Jazz',
       'Country/Folk', 'Musical', 'Soundtrack', 'Experimental'],
      dtype=object)

In [ ]:
music_df.Genre.value_counts()


,count
Genre,
Unknown,147494
Electronic,3788
Pop,3613
Rock,3440
Alternative/Indie,3085
Hip-Hop/Rap,2064
Country/Folk,792
Metal,533
Blues/Jazz,316


In [ ]:
music_df.head()


,Unnamed: 0,Username,Artist,Track,Album,Date,Time,Genre
0,0,Babs_05,Isobel Campbell,The Circus Is Leaving Town,Ballad of the Broken Seas,31 Jan 2021,23:36,Alternative/Indie
1,1,Babs_05,Isobel Campbell,Dusty Wreath,Ballad of the Broken Seas,31 Jan 2021,23:32,Alternative/Indie
2,2,Babs_05,Isobel Campbell,Honey Child What Can I Do?,Ballad of the Broken Seas,31 Jan 2021,23:28,Alternative/Indie
3,3,Babs_05,Isobel Campbell,It's Hard To Kill A Bad Thing,Ballad of the Broken Seas,31 Jan 2021,23:25,Alternative/Indie
4,4,Babs_05,Isobel Campbell,Saturday's Gone,Ballad of the Broken Seas,31 Jan 2021,23:21,Alternative/Indie


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Cargar el dataset y dividir en conocidos y desconocidos
music_df = pd.read_csv('/content/drive/MyDrive/TFM/Datasets/Last.fm_data_with_genres.csv')
known_genres_df = music_df[music_df['Genre'] != 'Unknown']
unknown_genres_df = music_df[music_df['Genre'] == 'Unknown']

# Crear una columna de texto para representar la combinación de características
known_genres_df['text_features'] = (known_genres_df['Artist'] + ' ' + known_genres_df['Album'] + ' ' + known_genres_df['Track']).fillna('')
unknown_genres_df['text_features'] = (unknown_genres_df['Artist'] + ' ' + unknown_genres_df['Album'] + ' ' + unknown_genres_df['Track']).fillna('')

# Dividir en características y etiquetas para entrenamiento
X = known_genres_df['text_features']
y = known_genres_df['Genre']

# Definir el pipeline con un modelo de RandomForest optimizado
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=1000, stop_words='english')),  # Ajustar palabras clave representativas
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
])

# Validación cruzada
scores = cross_val_score(pipeline, X, y, cv=5, scoring='accuracy')
print("Cross-validation accuracy scores:", scores)
print("Mean cross-validation accuracy:", scores.mean())

# Entrenar el modelo
pipeline.fit(X, y)

# Predecir géneros en el conjunto desconocido
unknown_genres_df['Predicted_Genre'] = pipeline.predict(unknown_genres_df['text_features'])

# Unir y guardar el resultado
music_df.loc[music_df['Genre'] == 'Unknown', 'Genre'] = unknown_genres_df['Predicted_Genre']
music_df.to_csv('/content/drive/MyDrive/TFM/Datasets/Last.fm_data_with_predicted_genres.csv', index=False)

# Mostrar la distribución de géneros después de la predicción
print("\nDistribución de géneros después de la predicción:")
print(music_df['Genre'].value_counts())


<ipython-input-56-577bf73a8c47>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  known_genres_df['text_features'] = (known_genres_df['Artist'] + ' ' + known_genres_df['Album'] + ' ' + known_genres_df['Track']).fillna('')
<ipython-input-56-577bf73a8c47>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unknown_genres_df['text_features'] = (unknown_genres_df['Artist'] + ' ' + unknown_genres_df['Album'] + ' ' + unknown_genres_df['Track']).fillna('')


Cross-validation accuracy scores: [0.99919614 0.99946409 1.         1.         0.99973198]
Mean cross-validation accuracy: 0.9996784422280461


<ipython-input-56-577bf73a8c47>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unknown_genres_df['Predicted_Genre'] = pipeline.predict(unknown_genres_df['text_features'])



Distribución de géneros después de la predicción:
Genre
Rock                 129961
Pop                   11041
Alternative/Indie      9633
Electronic             9113
Hip-Hop/Rap            2281
Country/Folk            972
Blues/Jazz              709
Musical                 586
Metal                   559
Soundtrack              517
Classical               473
Experimental            308
Name: count, dtype: int64


In [ ]:
# Muestra de registros originalmente desconocidos con el género predicho
import pandas as pd

# Seleccionar una muestra aleatoria de registros con género predicho
predicted_sample = unknown_genres_df[['Artist', 'Album', 'Track', 'Predicted_Genre']].sample(10, random_state=1)

# Mostrar la muestra para ver los resultados
print("Muestra de registros con géneros predichos:")
print(predicted_sample)


Muestra de registros con géneros predichos:
                                       Artist  \
143172                       Cabaret Voltaire   
124249                                  Queen   
106303  Little Baby Bum Nursery Rhyme Friends   
132975                            Still Woozy   
102912                      Charlie Brown Jr.   
30979                          Crate Classics   
71525        Ike Turner & The Kings Of Rhythm   
41723                                  Tennis   
146298                                 TyLuv.   
155574                      Mary Lou Williams   

                                       Album                          Track  \
143172                                  2x45                         Yashar   
124249     News of the World (2011 Remaster)  We Will Rock You - Remastered   
106303  50 Littlebabybum Nursery Rhyme Hits!    Bear Went Over the Mountain   
132975                                Window                         Window   
102912               

In [ ]:
# Contar las muestras por género en el conjunto de datos de entrenamiento
known_genres_distribution = known_genres_df['Genre'].value_counts()
print("Distribución de géneros en el conjunto de datos de entrenamiento:")
print(known_genres_distribution)

# Si observamos que algunos géneros están subrepresentados, podríamos tomar muestras adicionales
# Para ello, asegurémonos de tener un número similar de muestras en cada género.
# Supongamos que queremos al menos 500 ejemplos por género
min_samples = 500
for genre, count in known_genres_distribution.items():
    if count < min_samples:
        # Aumentar muestras en los géneros menos representados
        additional_samples = known_genres_df[known_genres_df['Genre'] == genre]
        additional_samples = additional_samples.sample(min_samples - count, replace=True)
        known_genres_df = pd.concat([known_genres_df, additional_samples])

# Verificar la nueva distribución
new_known_genres_distribution = known_genres_df['Genre'].value_counts()
print("Nueva distribución de géneros en el conjunto de datos de entrenamiento:")
print(new_known_genres_distribution)


Distribución de géneros en el conjunto de datos de entrenamiento:
Genre
Electronic           3788
Pop                  3613
Rock                 3440
Alternative/Indie    3085
Hip-Hop/Rap          2064
Country/Folk          792
Metal                 533
Blues/Jazz            316
Classical             295
Experimental          266
Musical               250
Soundtrack            217
Name: count, dtype: int64
Nueva distribución de géneros en el conjunto de datos de entrenamiento:
Genre
Electronic           3788
Pop                  3613
Rock                 3440
Alternative/Indie    3085
Hip-Hop/Rap          2064
Country/Folk          792
Metal                 533
Classical             500
Blues/Jazz            500
Musical               500
Soundtrack            500
Experimental          500
Name: count, dtype: int64


In [ ]:
# Balancear las clases seleccionando el mismo número de muestras por género
min_samples = 500  # Definido como mínimo de muestras para cada género
balanced_known_genres_df = known_genres_df.groupby('Genre').apply(lambda x: x.sample(min_samples)).reset_index(drop=True)

# Verificar la distribución después del balanceo
balanced_distribution = balanced_known_genres_df['Genre'].value_counts()
print("Distribución después del balanceo:")
print(balanced_distribution)


Distribución después del balanceo:
Genre
Alternative/Indie    500
Blues/Jazz           500
Classical            500
Country/Folk         500
Electronic           500
Experimental         500
Hip-Hop/Rap          500
Metal                500
Musical              500
Pop                  500
Rock                 500
Soundtrack           500
Name: count, dtype: int64


<ipython-input-59-97b6c5f3c644>:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  balanced_known_genres_df = known_genres_df.groupby('Genre').apply(lambda x: x.sample(min_samples)).reset_index(drop=True)


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Crear el pipeline de entrenamiento
pipeline_balanced = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
])

# Entrenar el modelo con el conjunto de datos balanceado
X_balanced = balanced_known_genres_df['text_features']
y_balanced = balanced_known_genres_df['Genre']
pipeline_balanced.fit(X_balanced, y_balanced)

# Evaluar el modelo con validación cruzada
cross_val_scores_balanced = cross_val_score(pipeline_balanced, X_balanced, y_balanced, cv=5)
print("Puntuaciones de validación cruzada:", cross_val_scores_balanced)
print("Precisión media de validación cruzada:", cross_val_scores_balanced.mean())


Puntuaciones de validación cruzada: [0.9975     1.         1.         1.         0.99916667]
Precisión media de validación cruzada: 0.9993333333333334


In [ ]:
# Comprobar las longitudes
num_unknown_genres = len(music_df[music_df['Genre'] == 'Unknown'])
num_predictions = len(unknown_genres_df['Predicted_Genre'])

print(f"Filas con género 'Unknown' en music_df: {num_unknown_genres}")
print(f"Número de predicciones generadas: {num_predictions}")

# Verificar si las longitudes coinciden
if num_unknown_genres == num_predictions:
    # Asignar los géneros predichos
    music_df.loc[music_df['Genre'] == 'Unknown', 'Genre'] = unknown_genres_df['Predicted_Genre'].values
else:
    print("Las longitudes no coinciden. Asegúrate de que los conjuntos de datos están alineados correctamente.")



Filas con género 'Unknown' en music_df: 0
Número de predicciones generadas: 147494
Las longitudes no coinciden. Asegúrate de que los conjuntos de datos están alineados correctamente.


In [ ]:
# Revisar algunos registros de music_df para verificar si quedan géneros 'Unknown'
print("Muestra de los primeros registros en music_df:")
print(music_df.head())

# Revisar la distribución final de géneros en music_df
genre_distribution = music_df['Genre'].value_counts()
print("\nDistribución final de géneros en music_df:")
print(genre_distribution)


Muestra de los primeros registros en music_df:
   Unnamed: 0 Username           Artist                          Track  \
0           0  Babs_05  Isobel Campbell     The Circus Is Leaving Town   
1           1  Babs_05  Isobel Campbell                   Dusty Wreath   
2           2  Babs_05  Isobel Campbell     Honey Child What Can I Do?   
3           3  Babs_05  Isobel Campbell  It's Hard To Kill A Bad Thing   
4           4  Babs_05  Isobel Campbell                Saturday's Gone   

                       Album         Date    Time              Genre  
0  Ballad of the Broken Seas  31 Jan 2021   23:36  Alternative/Indie  
1  Ballad of the Broken Seas  31 Jan 2021   23:32  Alternative/Indie  
2  Ballad of the Broken Seas  31 Jan 2021   23:28  Alternative/Indie  
3  Ballad of the Broken Seas  31 Jan 2021   23:25  Alternative/Indie  
4  Ballad of the Broken Seas  31 Jan 2021   23:21  Alternative/Indie  

Distribución final de géneros en music_df:
Genre
Rock                 129961
Pop 

In [ ]:
music_df.to_csv('/content/drive/MyDrive/TFM/Datasets/Last.fm_data_with_genres_completed.csv', index=False)


In [ ]:
music_df = pd.read_csv('/content/drive/MyDrive/TFM/Datasets/Last.fm_data_with_genres_completed.csv')

In [ ]:
music_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 166153 entries, 0 to 166152
Data columns (total 8 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  166153 non-null  int64 
 1   Username    166153 non-null  object
 2   Artist      166153 non-null  object
 3   Track       166153 non-null  object
 4   Album       166153 non-null  object
 5   Date        166153 non-null  object
 6   Time        166153 non-null  object
 7   Genre       166153 non-null  object
dtypes: int64(1), object(7)
memory usage: 10.1+ MB


In [ ]:
music_df.head()

,Unnamed: 0,Username,Artist,Track,Album,Date,Time,Genre
0,0,Babs_05,Isobel Campbell,The Circus Is Leaving Town,Ballad of the Broken Seas,31 Jan 2021,23:36,Alternative/Indie
1,1,Babs_05,Isobel Campbell,Dusty Wreath,Ballad of the Broken Seas,31 Jan 2021,23:32,Alternative/Indie
2,2,Babs_05,Isobel Campbell,Honey Child What Can I Do?,Ballad of the Broken Seas,31 Jan 2021,23:28,Alternative/Indie
3,3,Babs_05,Isobel Campbell,It's Hard To Kill A Bad Thing,Ballad of the Broken Seas,31 Jan 2021,23:25,Alternative/Indie
4,4,Babs_05,Isobel Campbell,Saturday's Gone,Ballad of the Broken Seas,31 Jan 2021,23:21,Alternative/Indie


In [ ]:
music_df.Genre.value_counts()

,count
Genre,
Rock,129961
Pop,11041
Alternative/Indie,9633
Electronic,9113
Hip-Hop/Rap,2281
Country/Folk,972
Blues/Jazz,709
Musical,586
Metal,559


In [ ]:
# Definir el número de muestras deseado para el género 'Rock' para equilibrar con otros géneros
target_sample_size = 20000  # Ajusta este número según el balance deseado

# Crear una muestra del género 'Rock' para reducir su tamaño
rock_sample = music_df[music_df['Genre'] == 'Rock'].sample(target_sample_size, random_state=42)

# Filtrar el dataset para excluir el exceso de 'Rock' y luego unirlo con la muestra
balanced_music_df = pd.concat([music_df[music_df['Genre'] != 'Rock'], rock_sample])

# Verificar la nueva distribución
balanced_distribution = balanced_music_df['Genre'].value_counts()
print("Distribución de géneros después de reducir 'Rock':")
print(balanced_distribution)


Distribución de géneros después de reducir 'Rock':
Genre
Rock                 20000
Pop                  11041
Alternative/Indie     9633
Electronic            9113
Hip-Hop/Rap           2281
Country/Folk           972
Blues/Jazz             709
Musical                586
Metal                  559
Soundtrack             517
Classical              473
Experimental           308
Name: count, dtype: int64


In [ ]:
# Guardar el dataset equilibrado
music_df.to_csv('/content/drive/MyDrive/TFM/Datasets/musica_integracionv2.csv', index=False)


In [ ]:
music_df.Genre.value_counts()

,count
Genre,
Rock,129961
Pop,11041
Alternative/Indie,9633
Electronic,9113
Hip-Hop/Rap,2281
Country/Folk,972
Blues/Jazz,709
Musical,586
Metal,559


Después de realizar la clasificación el dataset queda listo pero se tienen demasiados registros de rock, lo q podría llegar a sesgar el modelo de manera que se decide tomar una muestra para balancear los datos

In [ ]:
# Reducir el número de registros en el género 'Rock' y equilibrar la distribución
target_sample_size = 20000  # Ajusta el número según el balance deseado
rock_sample = music_df[music_df['Genre'] == 'Rock'].sample(target_sample_size, random_state=42)
balanced_music_df = pd.concat([music_df[music_df['Genre'] != 'Rock'], rock_sample])

# Verificar la nueva distribución antes de guardar
print("Distribución de géneros después de reducir 'Rock':")
print(balanced_music_df['Genre'].value_counts())

# Guardar el dataset balanceado
output_path = '/content/drive/MyDrive/TFM/Datasets/musica_integracionv2.csv'
balanced_music_df.to_csv(output_path, index=False)

print(f"Dataset guardado correctamente en {output_path}")


Distribución de géneros después de reducir 'Rock':
Genre
Rock                 20000
Pop                  11041
Alternative/Indie     9633
Electronic            9113
Hip-Hop/Rap           2281
Country/Folk           972
Blues/Jazz             709
Musical                586
Metal                  559
Soundtrack             517
Classical              473
Experimental           308
Name: count, dtype: int64
Dataset guardado correctamente en /content/drive/MyDrive/TFM/Datasets/musica_integracionv2.csv


In [ ]:
music_df = pd.read_csv('/content/drive/MyDrive/TFM/Datasets/musica_integracionv2.csv')

In [ ]:
music_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56192 entries, 0 to 56191
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  56192 non-null  int64 
 1   Username    56192 non-null  object
 2   Artist      56192 non-null  object
 3   Track       56192 non-null  object
 4   Album       56192 non-null  object
 5   Date        56192 non-null  object
 6   Time        56192 non-null  object
 7   Genre       56192 non-null  object
dtypes: int64(1), object(7)
memory usage: 3.4+ MB


In [ ]:
music_df.Genre.value_counts()

,count
Genre,
Rock,20000
Pop,11041
Alternative/Indie,9633
Electronic,9113
Hip-Hop/Rap,2281
Country/Folk,972
Blues/Jazz,709
Musical,586
Metal,559


In [ ]:
music_df.Genre.unique()

array(['Alternative/Indie', 'Electronic', 'Pop', 'Hip-Hop/Rap',
       'Classical', 'Soundtrack', 'Blues/Jazz', 'Musical', 'Experimental',
       'Metal', 'Country/Folk', 'Rock'], dtype=object)

In [ ]:
music_df.head()

,Unnamed: 0,Username,Artist,Track,Album,Date,Time,Genre
0,0,Babs_05,Isobel Campbell,The Circus Is Leaving Town,Ballad of the Broken Seas,31 Jan 2021,23:36,Alternative/Indie
1,1,Babs_05,Isobel Campbell,Dusty Wreath,Ballad of the Broken Seas,31 Jan 2021,23:32,Alternative/Indie
2,2,Babs_05,Isobel Campbell,Honey Child What Can I Do?,Ballad of the Broken Seas,31 Jan 2021,23:28,Alternative/Indie
3,3,Babs_05,Isobel Campbell,It's Hard To Kill A Bad Thing,Ballad of the Broken Seas,31 Jan 2021,23:25,Alternative/Indie
4,4,Babs_05,Isobel Campbell,Saturday's Gone,Ballad of the Broken Seas,31 Jan 2021,23:21,Alternative/Indie


Ahora se realizarán verificaciones para determinar si la clasificación de artista y género si es coherente

In [ ]:
import pandas as pd

# Cargar el dataset equilibrado
balanced_music_df = pd.read_csv('/content/drive/MyDrive/TFM/Datasets/musica_integracionv2.csv')

# Seleccionar registros aleatorios para verificar
sample_size = 10  # Puedes ajustar el número de muestras que deseas revisar
sample_records = balanced_music_df.sample(sample_size, random_state=42)

# Mostrar los registros seleccionados
print("Muestra de registros para verificar consistencia:")
print(sample_records[['Artist', 'Album', 'Track', 'Genre']])

# Verificar la distribución general para asegurarse de que el balance sigue siendo adecuado
print("\nDistribución de géneros en el dataset balanceado:")
print(balanced_music_df['Genre'].value_counts())


Muestra de registros para verificar consistencia:
                 Artist                                              Album  \
18674              ABBA                              Live at Wembley Arena   
43473         Chris Rea  Dancing with Strangers (Deluxe Edition, 2019 R...   
19242             Bicep                                              Isles   
27582         Mushkilla                                           RMX Dubs   
5246   Boards of Canada                             The Campfire Headphase   
11505      Eddie Vedder                Live from Nowhere Near You, Vol. II   
23927        The Weeknd                          Beauty Behind the Madness   
49714              Goat                                            Requiem   
53155       Limp Bizkit  Chocolate Starfish and the Hot Dog Flavored Water   
3758             *shels                       Plains of the Purple Buffalo   

                                            Track              Genre  
18674               

In [ ]:
import pandas as pd

# Cargar el archivo de música balanceado
balanced_music_df = pd.read_csv('/content/drive/MyDrive/TFM/Datasets/musica_integracionv2.csv')

# Seleccionar una muestra aleatoria de registros para revisar
sample_size = 10  # Puedes ajustar el tamaño de la muestra
sample_df = balanced_music_df.sample(sample_size, random_state=42)

# Mostrar la muestra aleatoria de registros
print("Muestra de registros para verificar coherencia:")
print(sample_df[['Artist', 'Album', 'Track', 'Genre']])


Muestra de registros para verificar coherencia:
                 Artist                                              Album  \
18674              ABBA                              Live at Wembley Arena   
43473         Chris Rea  Dancing with Strangers (Deluxe Edition, 2019 R...   
19242             Bicep                                              Isles   
27582         Mushkilla                                           RMX Dubs   
5246   Boards of Canada                             The Campfire Headphase   
11505      Eddie Vedder                Live from Nowhere Near You, Vol. II   
23927        The Weeknd                          Beauty Behind the Madness   
49714              Goat                                            Requiem   
53155       Limp Bizkit  Chocolate Starfish and the Hot Dog Flavored Water   
3758             *shels                       Plains of the Purple Buffalo   

                                            Track              Genre  
18674               Th

Ahora vamos con el data set de Netflix

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/TFM/Datasets/netflix_titles.csv')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7787 entries, 0 to 7786
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       7787 non-null   object
 1   type          7787 non-null   object
 2   title         7787 non-null   object
 3   director      5398 non-null   object
 4   cast          7069 non-null   object
 5   country       7280 non-null   object
 6   date_added    7777 non-null   object
 7   release_year  7787 non-null   int64 
 8   rating        7780 non-null   object
 9   duration      7787 non-null   object
 10  listed_in     7787 non-null   object
 11  description   7787 non-null   object
dtypes: int64(1), object(11)
memory usage: 730.2+ KB


In [ ]:
import pandas as pd

# Cargar el dataset de Netflix para exploración
netflix_df = pd.read_csv('/content/drive/MyDrive/TFM/Datasets/netflix_titles.csv')

# Explorar la estructura y el contenido inicial del dataset de Netflix
netflix_info = netflix_df.info()
netflix_head = netflix_df.head()
unique_columns = netflix_df.nunique()  # Verificar valores únicos en cada columna

netflix_info, netflix_head, unique_columns


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7787 entries, 0 to 7786
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       7787 non-null   object
 1   type          7787 non-null   object
 2   title         7787 non-null   object
 3   director      5398 non-null   object
 4   cast          7069 non-null   object
 5   country       7280 non-null   object
 6   date_added    7777 non-null   object
 7   release_year  7787 non-null   int64 
 8   rating        7780 non-null   object
 9   duration      7787 non-null   object
 10  listed_in     7787 non-null   object
 11  description   7787 non-null   object
dtypes: int64(1), object(11)
memory usage: 730.2+ KB


(None,
   show_id     type  title           director  \
 0      s1  TV Show     3%                NaN   
 1      s2    Movie   7:19  Jorge Michel Grau   
 2      s3    Movie  23:59       Gilbert Chan   
 3      s4    Movie      9        Shane Acker   
 4      s5    Movie     21     Robert Luketic   
 
                                                 cast        country  \
 0  João Miguel, Bianca Comparato, Michel Gomes, R...         Brazil   
 1  Demián Bichir, Héctor Bonilla, Oscar Serrano, ...         Mexico   
 2  Tedd Chan, Stella Chung, Henley Hii, Lawrence ...      Singapore   
 3  Elijah Wood, John C. Reilly, Jennifer Connelly...  United States   
 4  Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...  United States   
 
           date_added  release_year rating   duration  \
 0    August 14, 2020          2020  TV-MA  4 Seasons   
 1  December 23, 2016          2016  TV-MA     93 min   
 2  December 20, 2018          2011      R     78 min   
 3  November 16, 2017          2009

In [ ]:
df.type.unique()

array(['TV Show', 'Movie'], dtype=object)

In [ ]:
df.cast.unique()

array(['João Miguel, Bianca Comparato, Michel Gomes, Rodolfo Valente, Vaneza Oliveira, Rafael Lozano, Viviane Porto, Mel Fronckowiak, Sergio Mamberti, Zezé Motta, Celso Frateschi',
       'Demián Bichir, Héctor Bonilla, Oscar Serrano, Azalia Ortiz, Octavio Michel, Carmen Beato',
       'Tedd Chan, Stella Chung, Henley Hii, Lawrence Koh, Tommy Kuan, Josh Lai, Mark Lee, Susan Leong, Benjamin Lim',
       ...,
       'Vicky Kaushal, Sarah-Jane Dias, Raaghav Chanana, Manish Chaudhary, Meghna Malik, Malkeet Rauni, Anita Shabdish, Chittaranjan Tripathy',
       'Nasty C', 'Adriano Zumbo, Rachel Khoo'], dtype=object)

In [ ]:
df.rating.unique()

array(['TV-MA', 'R', 'PG-13', 'TV-14', 'TV-PG', 'NR', 'TV-G', 'TV-Y', nan,
       'TV-Y7', 'PG', 'G', 'NC-17', 'TV-Y7-FV', 'UR'], dtype=object)

In [ ]:
df.listed_in.unique()

array(['International TV Shows, TV Dramas, TV Sci-Fi & Fantasy',
       'Dramas, International Movies',
       'Horror Movies, International Movies',
       'Action & Adventure, Independent Movies, Sci-Fi & Fantasy',
       'Dramas', 'International TV Shows, TV Dramas, TV Mysteries',
       'Horror Movies, International Movies, Thrillers',
       'Dramas, Thrillers',
       'Crime TV Shows, International TV Shows, TV Dramas',
       'Crime TV Shows, Docuseries, International TV Shows',
       'Documentaries, International Movies, Sports Movies',
       'Independent Movies, Sci-Fi & Fantasy, Thrillers',
       'Dramas, International Movies, Thrillers',
       'International TV Shows, TV Dramas',
       'Comedies, Dramas, Independent Movies', 'Sports Movies',
       'Dramas, Independent Movies, International Movies',
       'Action & Adventure, Dramas, International Movies',
       'Anime Series, International TV Shows', 'Documentaries',
       'Reality TV', 'Documentaries, International

In [ ]:
import pandas as pd

# Cargar el dataset de Netflix
netflix_df = pd.read_csv('/content/drive/MyDrive/TFM/Datasets/netflix_titles.csv')

# Verificar valores nulos en cada columna
null_counts = netflix_df.isnull().sum()
print("Valores nulos por columna:")
print(null_counts)

# Visualizar filas con valores nulos en las columnas principales (ejemplo: director, cast, country, date_added, rating)
columns_with_nulls = ['director', 'cast', 'country', 'date_added', 'rating']
print("\nEjemplos de filas con valores nulos en columnas clave:")
for column in columns_with_nulls:
    print(f"\nFilas con valores nulos en la columna '{column}':")
    print(netflix_df[netflix_df[column].isnull()].head())


Valores nulos por columna:
show_id            0
type               0
title              0
director        2389
cast             718
country          507
date_added        10
release_year       0
rating             7
duration           0
listed_in          0
description        0
dtype: int64

Ejemplos de filas con valores nulos en columnas clave:

Filas con valores nulos en la columna 'director':
   show_id     type                                title director  \
0       s1  TV Show                                   3%      NaN   
11     s12  TV Show                                 1983      NaN   
16     s17  TV Show                               Feb-09      NaN   
19     s20    Movie                                  '89      NaN   
24     s25  TV Show  ​SAINT SEIYA: Knights of the Zodiac      NaN   

                                                 cast                country  \
0   João Miguel, Bianca Comparato, Michel Gomes, R...                 Brazil   
11  Robert Więckiewicz, Ma

In [ ]:
import pandas as pd

# Cargar el archivo
netflix_df = pd.read_csv('/content/drive/MyDrive/TFM/Datasets/netflix_titles.csv')

# 1. Llenar valores nulos en 'director' y 'cast' con 'Desconocido'
netflix_df['director'].fillna('Desconocido', inplace=True)
netflix_df['cast'].fillna('Desconocido', inplace=True)

# 2. Llenar valores nulos en 'country' con 'País Desconocido'
netflix_df['country'].fillna('País Desconocido', inplace=True)

# 3. Eliminar filas con valores nulos en 'date_added' y 'rating'
netflix_df.dropna(subset=['date_added', 'rating'], inplace=True)

# Verificar el resultado después de las modificaciones
print("Valores nulos por columna después de los cambios:")
print(netflix_df.isnull().sum())

# Guardar el dataset modificado
netflix_df.to_csv('/content/drive/MyDrive/TFM/Datasets/netflix_titles_cleaned.csv', index=False)


Valores nulos por columna después de los cambios:
show_id         0
type            0
title           0
director        0
cast            0
country         0
date_added      0
release_year    0
rating          0
duration        0
listed_in       0
description     0
dtype: int64


<ipython-input-92-ccf7fb7f1851>:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  netflix_df['director'].fillna('Desconocido', inplace=True)
<ipython-input-92-ccf7fb7f1851>:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)'

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/TFM/Datasets/netflix_titles_cleaned.csv')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7770 entries, 0 to 7769
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       7770 non-null   object
 1   type          7770 non-null   object
 2   title         7770 non-null   object
 3   director      7770 non-null   object
 4   cast          7770 non-null   object
 5   country       7770 non-null   object
 6   date_added    7770 non-null   object
 7   release_year  7770 non-null   int64 
 8   rating        7770 non-null   object
 9   duration      7770 non-null   object
 10  listed_in     7770 non-null   object
 11  description   7770 non-null   object
dtypes: int64(1), object(11)
memory usage: 728.6+ KB


In [ ]:
df.rating.unique()

array(['TV-MA', 'R', 'PG-13', 'TV-14', 'TV-PG', 'NR', 'TV-G', 'TV-Y',
       'TV-Y7', 'PG', 'G', 'NC-17', 'TV-Y7-FV', 'UR'], dtype=object)

In [ ]:
# Renombrar la columna 'rating' a 'classification_age'
netflix_df.rename(columns={'rating': 'classification_age'}, inplace=True)

# Verificar el cambio
print("Columnas después del renombrado:")
print(netflix_df.columns)


Columnas después del renombrado:
Index(['show_id', 'type', 'title', 'director', 'cast', 'country', 'date_added',
       'release_year', 'classification_age', 'duration', 'listed_in',
       'description'],
      dtype='object')


In [ ]:
df.listed_in.unique()

array(['International TV Shows, TV Dramas, TV Sci-Fi & Fantasy',
       'Dramas, International Movies',
       'Horror Movies, International Movies',
       'Action & Adventure, Independent Movies, Sci-Fi & Fantasy',
       'Dramas', 'International TV Shows, TV Dramas, TV Mysteries',
       'Horror Movies, International Movies, Thrillers',
       'Dramas, Thrillers',
       'Crime TV Shows, International TV Shows, TV Dramas',
       'Crime TV Shows, Docuseries, International TV Shows',
       'Documentaries, International Movies, Sports Movies',
       'Independent Movies, Sci-Fi & Fantasy, Thrillers',
       'Dramas, International Movies, Thrillers',
       'International TV Shows, TV Dramas',
       'Comedies, Dramas, Independent Movies', 'Sports Movies',
       'Dramas, Independent Movies, International Movies',
       'Action & Adventure, Dramas, International Movies',
       'Anime Series, International TV Shows', 'Documentaries',
       'Reality TV', 'Documentaries, International

In [ ]:
import pandas as pd

# Cargar el dataset de Netflix
netflix_df = pd.read_csv('/content/drive/MyDrive/TFM/Datasets/netflix_titles_cleaned.csv')

# Definir el mapeo de palabras clave a los géneros específicos que ya has definido
genre_mapping = {
    'Drama': 'Drama',
    'Comedy': 'Comedy',
    'Horror': 'Horror',
    'Action': 'Action',
    'Adventure': 'Action',  # Combinamos Adventure en Action
    'Fantasy': 'Fantasy',
    'Science Fiction': 'Science Fiction',
    'Sci-Fi': 'Science Fiction',
    'Romance': 'Romance',
    'Thriller': 'Thriller',
    'Mystery': 'Mystery',
    'Crime': 'Crime',
    'Documentary': 'Documentary',
    'Musical': 'Musical',
    'Music': 'Musical',
    'Independent': 'Independent',
    'International': 'International',
    'Family': 'Family',
    'Kids': 'Family',
    'Faith & Spirituality': 'Faith & Spirituality',
    'Stand-Up Comedy': 'Stand-Up Comedy',
    'Classic': 'Classic'
}

# Función para asignar género basado en las palabras clave en 'listed_in'
def map_genre(listed_in):
    for keyword, genre in genre_mapping.items():
        if keyword in listed_in:
            return genre
    return 'Other'  # Para categorías que no coinciden

# Aplicar el mapeo a la columna 'listed_in' para crear una nueva columna 'Genre'
netflix_df['Genre'] = netflix_df['listed_in'].apply(map_genre)

# Verificar la distribución de géneros después del mapeo
genre_distribution = netflix_df['Genre'].value_counts()
print("Distribución de géneros en Netflix después del mapeo:")
print(genre_distribution)



Distribución de géneros en Netflix después del mapeo:
Genre
Drama                   2808
International           1074
Other                   1051
Family                   753
Action                   596
Comedy                   381
Horror                   352
Crime                    223
Musical                  223
Thriller                 152
Fantasy                   66
Independent               59
Classic                   23
Faith & Spirituality       9
Name: count, dtype: int64


In [ ]:
import random

# Mostrar la distribución de géneros después del mapeo
print("Distribución de géneros en Netflix después del mapeo:")
print(genre_distribution)

# Muestra aleatoria de registros para verificar consistencia
def mostrar_muestras_por_genero(df, genero, muestras=5):
    muestras_genero = df[df['Genre'] == genero].sample(min(muestras, len(df[df['Genre'] == genero])), random_state=42)
    print(f"\nMuestras de '{genero}':")
    print(muestras_genero[['title', 'listed_in', 'Genre']])

# Revisar muestras de los géneros principales
generos_principales = genre_distribution.index.tolist()
for genero in generos_principales:
    mostrar_muestras_por_genero(netflix_df, genero, muestras=5)

# Muestra específica de "Other" para revisar si es necesario eliminar o remapear
mostrar_muestras_por_genero(netflix_df, 'Other', muestras=10)


Distribución de géneros en Netflix después del mapeo:
Genre
Drama                   2808
International           1074
Other                   1051
Family                   753
Action                   596
Comedy                   381
Horror                   352
Crime                    223
Musical                  223
Thriller                 152
Fantasy                   66
Independent               59
Classic                   23
Faith & Spirituality       9
Name: count, dtype: int64

Muestras de 'Drama':
                       title  \
4251  Much Ado About Nothing   
3218                     Joy   
7510     Welcome to New York   
1989          Emma' (Mother)   
1886                    Dude   

                                             listed_in  Genre  
4251  Dramas, Independent Movies, International Movies  Drama  
3218  Dramas, Independent Movies, International Movies  Drama  
7510                        Dramas, Independent Movies  Drama  
1989                      Dramas, Int

In [ ]:
import pandas as pd

# Cargar el dataset de Netflix
netflix_df = pd.read_csv('/content/drive/MyDrive/TFM/Datasets/netflix_titles_cleaned.csv')

# Definir el mapeo inicial con ajustes adicionales
genre_mapping = {
    'Drama': 'Drama',
    'Comedy': 'Comedy',
    'Horror': 'Horror',
    'Action': 'Action',
    'Adventure': 'Action',
    'Fantasy': 'Fantasy',
    'Science Fiction': 'Science Fiction',
    'Sci-Fi': 'Science Fiction',
    'Romance': 'Romance',
    'Thriller': 'Thriller',
    'Mystery': 'Mystery',
    'Crime': 'Crime',
    'Documentary': 'Documentary',
    'Musical': 'Musical',
    'Music': 'Musical',
    'Independent': 'Independent',
    'International': 'International',
    'Family': 'Family',
    'Kids': 'Family',
    'Faith & Spirituality': 'Faith & Spirituality',
    'Stand-Up Comedy': 'Comedy',  # Mapeo de Stand-Up Comedy en Comedy
    'Classic': 'Classic'
}

# Función para asignar géneros con refinamiento adicional
def map_genre(listed_in):
    # Verificar si algún género en genre_mapping coincide con 'listed_in'
    for keyword, genre in genre_mapping.items():
        if keyword in listed_in:
            return genre
    # Revisar patrones adicionales para casos ambiguos y asignar categorías más específicas
    if 'Documentaries' in listed_in or 'Docuseries' in listed_in:
        return 'Documentary'
    if 'Romantic' in listed_in and 'Drama' in listed_in:
        return 'Romance'
    if 'TV Shows' in listed_in or 'Reality TV' in listed_in:
        return 'International'
    # Retornar "Other" si no se encuentra una categoría específica
    return 'Other'

# Aplicar la función a la columna 'listed_in'
netflix_df['Genre'] = netflix_df['listed_in'].apply(map_genre)

# Verificar la distribución de géneros después del ajuste
final_genre_distribution = netflix_df['Genre'].value_counts()
print("Distribución de géneros en Netflix después del ajuste adicional:")
print(final_genre_distribution)

# Muestras aleatorias de géneros para verificar la coherencia
for genre in final_genre_distribution.index:
    print(f"\nMuestras de '{genre}':")
    print(netflix_df[netflix_df['Genre'] == genre][['title', 'listed_in', 'Genre']].sample(5))


Distribución de géneros en Netflix después del ajuste adicional:
Genre
Drama                   2808
International           1226
Family                   753
Action                   596
Documentary              586
Comedy                   381
Horror                   352
Other                    313
Crime                    223
Musical                  223
Thriller                 152
Fantasy                   66
Independent               59
Classic                   23
Faith & Spirituality       9
Name: count, dtype: int64

Muestras de 'Drama':
                              title  \
2554                      Guzaarish   
3757              Love Off the Cuff   
2422  Girlfriends' Guide to Divorce   
2356           Garuda in My Heart 2   
5210                      Riverdale   

                                            listed_in  Genre  
2554                     Dramas, International Movies  Drama  
3757           Comedies, Dramas, International Movies  Drama  
2422                  

In [ ]:
import pandas as pd

# Guardar el dataset refinado con el mapeo de géneros actualizado
netflix_df.to_csv('/content/drive/MyDrive/TFM/Datasets/netflix_titles_refined.csv', index=False)
print("Cambios guardados en el archivo 'netflix_titles_refined.csv'.")

# Cargar el dataset refinado guardado
refined_df = pd.read_csv('/content/drive/MyDrive/TFM/Datasets/netflix_titles_refined.csv')

# Función para mostrar una muestra aleatoria de registros de cada género
def show_random_samples(df, num_samples=5):
    genres = df['Genre'].unique()
    samples = {}
    for genre in genres:
        # Filtrar por género y tomar una muestra aleatoria
        genre_sample = df[df['Genre'] == genre].sample(min(num_samples, len(df[df['Genre'] == genre])), random_state=42)
        samples[genre] = genre_sample[['title', 'listed_in', 'Genre']]

    # Imprimir las muestras de cada género
    for genre, sample in samples.items():
        print(f"\nMuestra de registros para el género '{genre}':")
        print(sample)

# Llamar a la función para revisar muestras aleatorias
show_random_samples(refined_df)


Cambios guardados en el archivo 'netflix_titles_refined.csv'.

Muestra de registros para el género 'Drama':
                        title  \
4431                   Neruda   
6565  The Legend of Bruce Lee   
6624    The Many Faces of Ito   
4250                Mubarakan   
3044                    Iqbal   

                                              listed_in  Genre  
4431                       Dramas, International Movies  Drama  
6565  International TV Shows, TV Action & Adventure,...  Drama  
6624  International TV Shows, Romantic TV Shows, TV ...  Drama  
4250             Comedies, Dramas, International Movies  Drama  
3044   Dramas, Independent Movies, International Movies  Drama  

Muestra de registros para el género 'Horror':
                 title                                       listed_in   Genre
5575           Shutter             Horror Movies, International Movies  Horror
2593          Hannibal           TV Horror, TV Mysteries, TV Thrillers  Horror
1479  Conjuring Spi

In [ ]:
refined_df = pd.read_csv('/content/drive/MyDrive/TFM/Datasets/netflix_titles_refined.csv')

In [ ]:
refined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7770 entries, 0 to 7769
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       7770 non-null   object
 1   type          7770 non-null   object
 2   title         7770 non-null   object
 3   director      7770 non-null   object
 4   cast          7770 non-null   object
 5   country       7770 non-null   object
 6   date_added    7770 non-null   object
 7   release_year  7770 non-null   int64 
 8   rating        7770 non-null   object
 9   duration      7770 non-null   object
 10  listed_in     7770 non-null   object
 11  description   7770 non-null   object
 12  Genre         7770 non-null   object
dtypes: int64(1), object(12)
memory usage: 789.3+ KB


In [ ]:
refined_df.type.unique()

array(['TV Show', 'Movie'], dtype=object)

In [ ]:
refined_df.description.unique()

array(['In a future where the elite inhabit an island paradise far from the crowded slums, you get one chance to join the 3% saved from squalor.',
       'After a devastating earthquake hits Mexico City, trapped survivors from all walks of life wait to be rescued while trying desperately to stay alive.',
       "When an army recruit is found dead, his fellow soldiers are forced to confront a terrifying secret that's haunting their jungle island training camp.",
       ...,
       "In this documentary, South African rapper Nasty C hits the stage and streets of Tokyo, introducing himself to the city's sights, sounds and culture.",
       'Dessert wizard Adriano Zumbo looks for the next “Willy Wonka” in this tense competition that finds skilled amateurs competing for a $100,000 prize.',
       'This documentary delves into the mystique behind the blues-rock trio and explores how the enigmatic band created their iconic look and sound.'],
      dtype=object)

In [ ]:
# Filtrar el dataset de Netflix para eliminar los registros con el género 'Other'
netflix_df_refined = netflix_df[netflix_df['Genre'] != 'Other']

# Verificar la nueva distribución de géneros después de eliminar 'Other'
final_genre_distribution = netflix_df_refined['Genre'].value_counts()
print("Distribución de géneros en Netflix después de eliminar 'Other':")
print(final_genre_distribution)

# Guardar el dataset refinado sin 'Other' para la integración
netflix_df_refined.to_csv('/content/drive/MyDrive/TFM/Datasets/netflix_integracionv3.csv', index=False)


Distribución de géneros en Netflix después de eliminar 'Other':
Genre
Drama                   2808
International           1226
Family                   753
Action                   596
Documentary              586
Comedy                   381
Horror                   352
Crime                    223
Musical                  223
Thriller                 152
Fantasy                   66
Independent               59
Classic                   23
Faith & Spirituality       9
Name: count, dtype: int64


In [ ]:
import pandas as pd

# Cargar los datasets
movies_df = pd.read_csv('/content/drive/MyDrive/TFM/Datasets/integracionv1.csv')
netflix_df = pd.read_csv('/content/drive/MyDrive/TFM/Datasets/netflix_integracionv3.csv')
music_df = pd.read_csv('/content/drive/MyDrive/TFM/Datasets/musica_integracionv2.csv')

# Establecer el tipo de contenido
movies_df['type'] = 'Movie'
netflix_df['type'] = netflix_df['type'].replace({'TV Show': 'TV Show', 'Movie': 'Movie'})
music_df['type'] = 'Music'

# Convertir los títulos a minúsculas para facilitar la búsqueda de coincidencias
movies_df['title_lower'] = movies_df['title'].str.lower()
netflix_df['title_lower'] = netflix_df['title'].str.lower()
music_df['Track_lower'] = music_df['Track'].str.lower()

# Buscar coincidencias de títulos entre películas y Netflix
movie_netflix_matches = pd.merge(movies_df, netflix_df, left_on='title_lower', right_on='title_lower', suffixes=('_movie', '_netflix'))
print("Coincidencias entre títulos de películas y Netflix:")
print(movie_netflix_matches[['title_movie', 'title_netflix']].head())

# Buscar coincidencias de títulos entre música y películas
music_movie_matches = pd.merge(music_df, movies_df, left_on='Track_lower', right_on='title_lower', suffixes=('_music', '_movie'))
print("\nCoincidencias entre títulos de música y películas:")
print(music_movie_matches[['Track', 'title']].head())

# Buscar coincidencias de títulos entre música y Netflix
music_netflix_matches = pd.merge(music_df, netflix_df, left_on='Track_lower', right_on='title_lower', suffixes=('_music', '_netflix'))
print("\nCoincidencias entre títulos de música y Netflix:")
print(music_netflix_matches[['Track', 'title']].head())


Coincidencias entre títulos de películas y Netflix:
  title_movie title_netflix
0     Sabrina       Sabrina
1     Sabrina       Sabrina
2     Sabrina       Sabrina
3     Sabrina       Sabrina
4     Sabrina       Sabrina

Coincidencias entre títulos de música y películas:
      Track     title
0  Revolver  Revolver
1  Revolver  Revolver
2  Revolver  Revolver
3  Revolver  Revolver
4  Revolver  Revolver

Coincidencias entre títulos de música y Netflix:
         Track        title
0         Easy         Easy
1  The Monster  The Monster
2     The Call     The Call
3         Hook         Hook
4      Chelsea      Chelsea


In [ ]:
import pandas as pd

# Cargar los datasets
movies_df = pd.read_csv('/content/drive/MyDrive/TFM/Datasets/integracionv1.csv')
netflix_df = pd.read_csv('/content/drive/MyDrive/TFM/Datasets/netflix_integracionv3.csv')
music_df = pd.read_csv('/content/drive/MyDrive/TFM/Datasets/musica_integracionv2.csv')

# Paso 1: Añadir columna 'source' para distinguir el origen
movies_df['source'] = 'movies'
netflix_df['source'] = 'netflix'
music_df['source'] = 'music'

# Paso 2: Añadir columna 'type' con los valores apropiados
movies_df['type'] = 'Movie'
netflix_df['type'] = netflix_df['type'].replace({'TV Show': 'TV Show', 'Movie': 'Movie'})
music_df['type'] = 'Music'

# Paso 3: Convertir títulos a minúsculas en una nueva columna para comparar fácilmente
movies_df['title_lower'] = movies_df['title'].str.lower()
netflix_df['title_lower'] = netflix_df['title'].str.lower()
music_df['title_lower'] = music_df['Track'].str.lower()  # Usamos 'Track' como título de la música

# Identificar y remover duplicados en base a la columna en minúsculas
movies_df = movies_df.drop_duplicates(subset=['title_lower'])
netflix_df = netflix_df.drop_duplicates(subset=['title_lower'])
music_df = music_df.drop_duplicates(subset=['title_lower'])

# Integrar los datasets
combined_df = pd.concat([movies_df, netflix_df, music_df], ignore_index=True)

# Guardar el archivo final con los ajustes
combined_df.to_csv('/content/drive/MyDrive/TFM/Datasets/final_integrated_dataset.csv', index=False)


In [ ]:
data_in = pd.read_csv('/content/drive/MyDrive/TFM/Datasets/final_integrated_dataset.csv')

<ipython-input-192-d0ae8078bd6c>:1: DtypeWarning: Columns (1,2,4,5,12,13,14,15,16,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  data_in = pd.read_csv('/content/drive/MyDrive/TFM/Datasets/final_integrated_dataset.csv')


In [ ]:
data_in.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38133 entries, 0 to 38132
Data columns (total 29 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   movieId       3585 non-null   float64
 1   title         11042 non-null  object 
 2   genres        3585 non-null   object 
 3   userId        3585 non-null   float64
 4   rating        11042 non-null  object 
 5   timestamp     3585 non-null   object 
 6   imdbId        3585 non-null   float64
 7   tmdbId        3585 non-null   float64
 8   year          3585 non-null   float64
 9   source        38133 non-null  object 
 10  type          38133 non-null  object 
 11  title_lower   38133 non-null  object 
 12  show_id       7457 non-null   object 
 13  director      7457 non-null   object 
 14  cast          7457 non-null   object 
 15  country       7457 non-null   object 
 16  date_added    7457 non-null   object 
 17  release_year  7457 non-null   float64
 18  duration      7457 non-nul

In [ ]:
import pandas as pd

# Cargar el dataset unificado
data_in = pd.read_csv('/content/drive/MyDrive/TFM/Datasets/final_integrated_dataset.csv')

# Eliminar columnas innecesarias
columns_to_drop = ['imdbId', 'tmdbId', 'listed_in', 'Unnamed: 0', 'Username', 'title_lower' ]
data_in.drop(columns=columns_to_drop, inplace=True)

# Guardar el archivo sin las columnas eliminadas
data_in.to_csv('/content/drive/MyDrive/TFM/Datasets/final_integrated_dataset.csv', index=False)

# Verificar la estructura después de la eliminación
data_in.info()


<ipython-input-194-d5f2f1f05357>:4: DtypeWarning: Columns (1,2,4,5,12,13,14,15,16,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  data_in = pd.read_csv('/content/drive/MyDrive/TFM/Datasets/final_integrated_dataset.csv')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38133 entries, 0 to 38132
Data columns (total 23 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   movieId       3585 non-null   float64
 1   title         11042 non-null  object 
 2   genres        3585 non-null   object 
 3   userId        3585 non-null   float64
 4   rating        11042 non-null  object 
 5   timestamp     3585 non-null   object 
 6   year          3585 non-null   float64
 7   source        38133 non-null  object 
 8   type          38133 non-null  object 
 9   show_id       7457 non-null   object 
 10  director      7457 non-null   object 
 11  cast          7457 non-null   object 
 12  country       7457 non-null   object 
 13  date_added    7457 non-null   object 
 14  release_year  7457 non-null   float64
 15  duration      7457 non-null   object 
 16  description   7457 non-null   object 
 17  Genre         34548 non-null  object 
 18  Artist        27091 non-nu

In [ ]:
data_in.Genre.unique()

array([nan, 'Drama', 'Horror', 'Action', 'Crime', 'International',
       'Fantasy', 'Documentary', 'Musical', 'Faith & Spirituality',
       'Thriller', 'Family', 'Independent', 'Comedy', 'Classic',
       'Alternative/Indie', 'Electronic', 'Pop', 'Hip-Hop/Rap',
       'Classical', 'Soundtrack', 'Blues/Jazz', 'Experimental', 'Metal',
       'Country/Folk', 'Rock'], dtype=object)

In [ ]:
data_in.genres.value_counts()

,count
genres,
Comedy,366
Drama,330
Comedy|Drama,329
Action|Drama,283
Drama|Suspense,268
Action|Comedy,263
Action|Science Fiction,167
Drama|Romance,161
Action|Suspense,154


In [ ]:
import pandas as pd

# Cargar el dataset consolidado
data_in = pd.read_csv('/content/drive/MyDrive/TFM/Datasets/final_integrated_dataset.csv')

# Unificar `genres` en `Genre`, reemplazando los valores nulos de `Genre` con los de `genres`
data_in['Genre'] = data_in['Genre'].combine_first(data_in['genres'])

# Eliminar la columna `genres` ya que la información está en `Genre`
data_in.drop(columns=['genres'], inplace=True)

# Guardar el dataset actualizado
data_in.to_csv('/content/drive/MyDrive/TFM/Datasets/final_integrated_dataset.csv', index=False)

print("Unificación completada y archivo guardado como 'final_integrated_dataset_updated.csv'")


<ipython-input-196-17056b85698d>:4: DtypeWarning: Columns (1,2,4,5,9,10,11,12,13,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  data_in = pd.read_csv('/content/drive/MyDrive/TFM/Datasets/final_integrated_dataset.csv')


Unificación completada y archivo guardado como 'final_integrated_dataset_updated.csv'


In [ ]:
data_in = pd.read_csv('/content/drive/MyDrive/TFM/Datasets/final_integrated_dataset.csv')

<ipython-input-197-d0ae8078bd6c>:1: DtypeWarning: Columns (1,3,4,8,9,10,11,12,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  data_in = pd.read_csv('/content/drive/MyDrive/TFM/Datasets/final_integrated_dataset.csv')


In [ ]:
data_in.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38133 entries, 0 to 38132
Data columns (total 22 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   movieId       3585 non-null   float64
 1   title         11042 non-null  object 
 2   userId        3585 non-null   float64
 3   rating        11042 non-null  object 
 4   timestamp     3585 non-null   object 
 5   year          3585 non-null   float64
 6   source        38133 non-null  object 
 7   type          38133 non-null  object 
 8   show_id       7457 non-null   object 
 9   director      7457 non-null   object 
 10  cast          7457 non-null   object 
 11  country       7457 non-null   object 
 12  date_added    7457 non-null   object 
 13  release_year  7457 non-null   float64
 14  duration      7457 non-null   object 
 15  description   7457 non-null   object 
 16  Genre         38133 non-null  object 
 17  Artist        27091 non-null  object 
 18  Track         27091 non-nu

In [ ]:
data_in.Genre.unique()

array(['Action|Children', 'Comedy|Romance', 'Comedy|Drama', 'Comedy',
       'Action|Suspense', 'Action', 'Comedy|Horror', 'Drama',
       'Action|Romance', 'Drama|Suspense', 'Drama|Romance',
       'Action|Comedy', 'Comedy|Suspense', 'Drama|Horror',
       'Drama|Science Fiction', 'Children|Drama', 'Action|Drama',
       'Science Fiction|Suspense', 'Action|Fantasy', 'Suspense',
       'Action|Science Fiction', 'Children|Comedy', 'Documentary',
       'Action|Documentary', 'Horror', 'Fantasy|Horror',
       'Horror|Science Fiction', 'Drama|Musical',
       'Comedy|Science Fiction', 'Drama|Fantasy', 'Horror|Suspense',
       'Action|Horror', 'Children', 'Children|Family', 'Action|Family',
       'Family', 'Comedy|Fantasy', 'Comedy|Musical', 'Musical', 'Romance',
       'Fantasy', 'Science Fiction', 'Comedy|Family', 'Crime',
       'International', 'Faith & Spirituality', 'Thriller', 'Independent',
       'Classic', 'Alternative/Indie', 'Electronic', 'Pop', 'Hip-Hop/Rap',
       'Classic

In [ ]:
import pandas as pd

# Cargar el dataset
data_in = pd.read_csv('/content/drive/MyDrive/TFM/Datasets/final_integrated_dataset.csv')

# Convertir cada valor en la columna 'Genre' en una lista de géneros individuales
data_in['Genre'] = data_in['Genre'].fillna('').apply(lambda x: x.split('|') if isinstance(x, str) else [])

# Verificar la conversión
print("Ejemplos de valores en 'Genre' después de la conversión a listas:")
print(data_in['Genre'].head(10).tolist())

# Guardar el dataset con la columna 'Genre' en formato de lista
data_in.to_csv('/content/drive/MyDrive/TFM/Datasets/final_integrated_dataset.csv', index=False)

print("Conversión completada y guardada en 'final_integrated_dataset.csv'.")



<ipython-input-201-c43d36d254c0>:4: DtypeWarning: Columns (1,3,4,8,9,10,11,12,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  data_in = pd.read_csv('/content/drive/MyDrive/TFM/Datasets/final_integrated_dataset.csv')


Ejemplos de valores en 'Genre' después de la conversión a listas:
[['Action', 'Children'], ['Action', 'Children'], ['Comedy', 'Romance'], ['Comedy', 'Drama'], ['Comedy'], ['Action', 'Suspense'], ['Comedy', 'Romance'], ['Action', 'Children'], ['Action'], ['Action', 'Suspense']]
Conversión completada y guardada en 'final_integrated_dataset.csv'.


In [ ]:
data_in.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38133 entries, 0 to 38132
Data columns (total 22 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   movieId       3585 non-null   float64
 1   title         11042 non-null  object 
 2   userId        3585 non-null   float64
 3   rating        11042 non-null  object 
 4   timestamp     3585 non-null   object 
 5   year          3585 non-null   float64
 6   source        38133 non-null  object 
 7   type          38133 non-null  object 
 8   show_id       7457 non-null   object 
 9   director      7457 non-null   object 
 10  cast          7457 non-null   object 
 11  country       7457 non-null   object 
 12  date_added    7457 non-null   object 
 13  release_year  7457 non-null   float64
 14  duration      7457 non-null   object 
 15  description   7457 non-null   object 
 16  Genre         38133 non-null  object 
 17  Artist        27091 non-null  object 
 18  Track         27091 non-nu

In [ ]:
# Obtener todos los géneros únicos de la columna 'Genre'
unique_genres = set(genre for sublist in data_in['Genre'] for genre in sublist)
print("Géneros únicos en la columna 'Genre':")
print(unique_genres)


Géneros únicos en la columna 'Genre':
{'Faith & Spirituality', 'Comedy', 'Documentary', 'Action', 'Blues/Jazz', 'Metal', 'Thriller', 'Classical', 'Suspense', 'Crime', 'Horror', 'Experimental', 'Country/Folk', 'Drama', 'Classic', 'Pop', 'Soundtrack', 'Family', 'Independent', 'Hip-Hop/Rap', 'Fantasy', 'Alternative/Indie', 'Musical', 'Children', 'International', 'Romance', 'Science Fiction', 'Electronic', 'Rock'}


In [ ]:
data_in.type.unique()

array(['Movie', 'TV Show', 'Music'], dtype=object)

In [ ]:
data_in.type.value_counts()

,count
type,
Music,27091
Movie,8714
TV Show,2328


In [ ]:
data_in.rating.unique()

array(['4.0', '3.0', '5.0', '1.0', '2.0', '1.5', '0.5', '3.5', '4.5',
       '2.5', 'TV-MA', 'R', 'PG-13', 'TV-14', 'NR', 'TV-G', 'TV-PG',
       'TV-Y7', 'PG', 'G', 'TV-Y', 'NC-17', 'TV-Y7-FV', 'UR', nan],
      dtype=object)

In [ ]:
import pandas as pd

# Cargar el dataset
data_in = pd.read_csv('/content/drive/MyDrive/TFM/Datasets/final_integrated_dataset.csv')

# Función para separar los puntajes y clasificaciones de edad
def split_rating(value):
    if value in ['TV-MA', 'R', 'PG-13', 'TV-14', 'NR', 'TV-G', 'TV-PG', 'TV-Y7', 'PG', 'G', 'TV-Y', 'NC-17', 'TV-Y7-FV', 'UR']:
        return None, value  # Clasificación de edad
    elif pd.notnull(value):
        return float(value), None  # Puntaje numérico
    return None, None

# Aplicar la función y crear las nuevas columnas
data_in[['rating', 'classification_age']] = data_in['rating'].apply(lambda x: pd.Series(split_rating(x)))

# Verificar si la separación fue exitosa mostrando valores únicos de cada columna
print("Valores únicos en 'rating' después de la separación:", data_in['rating'].unique())
print("Valores únicos en 'classification_age':", data_in['classification_age'].unique())

# Guardar el dataset actualizado
data_in.to_csv('/content/drive/MyDrive/TFM/Datasets/final_integrated_dataset2.csv', index=False)
print("Separación completada y guardada en 'final_integrated_dataset.csv'.")


<ipython-input-213-cba61a55c446>:4: DtypeWarning: Columns (1,3,4,8,9,10,11,12,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  data_in = pd.read_csv('/content/drive/MyDrive/TFM/Datasets/final_integrated_dataset.csv')


Valores únicos en 'rating' después de la separación: [4.  3.  5.  1.  2.  1.5 0.5 3.5 4.5 2.5 nan]
Valores únicos en 'classification_age': [nan 'TV-MA' 'R' 'PG-13' 'TV-14' 'NR' 'TV-G' 'TV-PG' 'TV-Y7' 'PG' 'G'
 'TV-Y' 'NC-17' 'TV-Y7-FV' 'UR' None]
Separación completada y guardada en 'final_integrated_dataset.csv'.


In [ ]:
data_in.rating.unique()

array([4. , 3. , 5. , 1. , 2. , 1.5, 0.5, 3.5, 4.5, 2.5, nan])

In [ ]:
data_in.classification_age.unique()

array([nan, 'TV-MA', 'R', 'PG-13', 'TV-14', 'NR', 'TV-G', 'TV-PG',
       'TV-Y7', 'PG', 'G', 'TV-Y', 'NC-17', 'TV-Y7-FV', 'UR', None],
      dtype=object)

In [ ]:
data_in.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38133 entries, 0 to 38132
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   movieId             3585 non-null   float64
 1   title               11042 non-null  object 
 2   userId              3585 non-null   float64
 3   rating              3585 non-null   float64
 4   timestamp           3585 non-null   object 
 5   year                3585 non-null   float64
 6   source              38133 non-null  object 
 7   type                38133 non-null  object 
 8   show_id             7457 non-null   object 
 9   director            7457 non-null   object 
 10  cast                7457 non-null   object 
 11  country             7457 non-null   object 
 12  date_added          7457 non-null   object 
 13  release_year        7457 non-null   float64
 14  duration            7457 non-null   object 
 15  description         7457 non-null   object 
 16  Genr

In [ ]:
data_in.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38133 entries, 0 to 38132
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   movieId             3585 non-null   float64
 1   title               11042 non-null  object 
 2   userId              3585 non-null   float64
 3   rating              38133 non-null  float64
 4   timestamp           3585 non-null   object 
 5   year                3585 non-null   float64
 6   source              38133 non-null  object 
 7   type                38133 non-null  object 
 8   show_id             7457 non-null   object 
 9   director            7457 non-null   object 
 10  cast                7457 non-null   object 
 11  country             7457 non-null   object 
 12  date_added          7457 non-null   object 
 13  release_year        7457 non-null   float64
 14  duration            7457 non-null   object 
 15  description         7457 non-null   object 
 16  Genr

In [ ]:
data_in.rating.unique()

array([4. , 3. , 5. , 1. , 2. , 1.5, 0.5, 3.5, 4.5, 2.5])

In [ ]:
import pandas as pd
import numpy as np

# Cargar el dataset actualizado
data_in = pd.read_csv('/content/drive/MyDrive/TFM/Datasets/final_integrated_dataset3.csv')

# Clasificaciones por edad sugeridas para géneros comunes
age_classifications = {
    'Horror': 'R',
    'Action': 'PG-13',
    'Comedy': 'PG',
    'Drama': 'PG-13',
    'Fantasy': 'PG',
    'Family': 'G',
    'Children': 'G',
    'Documentary': 'PG',
    'Crime': 'TV-14',
    'Thriller': 'R',
    'Science Fiction': 'PG-13',
    'Faith & Spirituality': 'G',
    'Musical': 'G',
    'Independent': 'PG-13',
    'Classic': 'PG'
}

# Función para asignar clasificación de edad basada en el género
def assign_age_classification(row):
    if isinstance(row['Genre'], list):  # Verificar que la columna sea una lista
        for genre in row['Genre']:
            if genre in age_classifications:
                return age_classifications[genre]
    return np.nan  # Si no coincide ningún género, se deja NaN

# Aplicar la función a las filas que tienen NaN en la columna de clasificación de edad
data_in['classification_age'] = data_in.apply(
    lambda x: assign_age_classification(x) if pd.isna(x['classification_age']) else x['classification_age'], axis=1
)

# Verificar los resultados y contar valores NaN restantes
remaining_nan = data_in['classification_age'].isna().sum()
print(f"Valores NaN restantes en 'classification_age': {remaining_nan}")

# Guardar el dataset actualizado
data_in.to_csv('/content/drive/MyDrive/TFM/Datasets/final_integrated_dataset4.csv', index=False)
print("Archivo guardado como 'final_integrated_dataset3.csv'")


<ipython-input-223-cc332ff67399>:5: DtypeWarning: Columns (1,4,8,9,10,11,12,14,15,22) have mixed types. Specify dtype option on import or set low_memory=False.
  data_in = pd.read_csv('/content/drive/MyDrive/TFM/Datasets/final_integrated_dataset3.csv')


Valores NaN restantes en 'classification_age': 30676
Archivo guardado como 'final_integrated_dataset3.csv'


In [ ]:
data_in = pd.read_csv('/content/drive/MyDrive/TFM/Datasets/final_integrated_dataset4.csv')

<ipython-input-266-00bbb228c004>:1: DtypeWarning: Columns (1,4,8,9,10,11,12,14,15,22) have mixed types. Specify dtype option on import or set low_memory=False.
  data_in = pd.read_csv('/content/drive/MyDrive/TFM/Datasets/final_integrated_dataset4.csv')


In [ ]:
data_in.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38133 entries, 0 to 38132
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   movieId             3585 non-null   float64
 1   title               11042 non-null  object 
 2   userId              3585 non-null   float64
 3   rating              38133 non-null  float64
 4   timestamp           3585 non-null   object 
 5   year                3585 non-null   float64
 6   source              38133 non-null  object 
 7   type                38133 non-null  object 
 8   show_id             7457 non-null   object 
 9   director            7457 non-null   object 
 10  cast                7457 non-null   object 
 11  country             7457 non-null   object 
 12  date_added          7457 non-null   object 
 13  release_year        7457 non-null   float64
 14  duration            7457 non-null   object 
 15  description         7457 non-null   object 
 16  Genr

Después de hacer muchas pruebas se descarta la opción de unificar los datasets, pues en realidad es muy dificil se tienen que asumir muchas cosas, y se va perdiendo la integridad de los datos

In [ ]:
import pandas as pd

# Cargar el dataset actual
data_in = pd.read_csv('/content/drive/MyDrive/TFM/Datasets/final_integrated_dataset4.csv')

# Filtrar por cada fuente y guardar los datasets resultantes
movies_df = data_in[data_in['source'] == 'movies']
netflix_df = data_in[data_in['source'] == 'netflix']
music_df = data_in[data_in['source'] == 'music']

# Guardar cada dataset separado
movies_df.to_csv('/content/drive/MyDrive/TFM/Datasets/movies_dataset.csv', index=False)
netflix_df.to_csv('/content/drive/MyDrive/TFM/Datasets/netflix_dataset.csv', index=False)
music_df.to_csv('/content/drive/MyDrive/TFM/Datasets/music_dataset.csv', index=False)

print("Archivos separados guardados como 'movies_dataset.csv', 'netflix_dataset.csv' y 'music_dataset.csv'")


<ipython-input-277-47038d7af9fb>:4: DtypeWarning: Columns (1,4,8,9,10,11,12,14,15,22) have mixed types. Specify dtype option on import or set low_memory=False.
  data_in = pd.read_csv('/content/drive/MyDrive/TFM/Datasets/final_integrated_dataset4.csv')


Archivos separados guardados como 'movies_dataset.csv', 'netflix_dataset.csv' y 'music_dataset.csv'


In [ ]:
movies = pd.read_csv('/content/drive/MyDrive/TFM/Datasets/movies_dataset.csv')
netflix  = pd.read_csv('/content/drive/MyDrive/TFM/Datasets/netflix_dataset.csv')
music  = pd.read_csv('/content/drive/MyDrive/TFM/Datasets/music_dataset.csv')

In [ ]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3585 entries, 0 to 3584
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   movieId             3585 non-null   float64
 1   title               3585 non-null   object 
 2   userId              3585 non-null   float64
 3   rating              3585 non-null   float64
 4   timestamp           3585 non-null   object 
 5   year                3585 non-null   float64
 6   source              3585 non-null   object 
 7   type                3585 non-null   object 
 8   show_id             0 non-null      float64
 9   director            0 non-null      float64
 10  cast                0 non-null      float64
 11  country             0 non-null      float64
 12  date_added          0 non-null      float64
 13  release_year        0 non-null      float64
 14  duration            0 non-null      float64
 15  description         0 non-null      float64
 16  Genre 

In [ ]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90267 entries, 0 to 90266
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   movieId    90267 non-null  int64  
 1   title      90267 non-null  object 
 2   genres     90267 non-null  object 
 3   userId     90267 non-null  int64  
 4   rating     90267 non-null  float64
 5   timestamp  90267 non-null  object 
 6   imdbId     90267 non-null  int64  
 7   tmdbId     90267 non-null  float64
 8   year       90267 non-null  int64  
dtypes: float64(2), int64(4), object(3)
memory usage: 6.2+ MB


In [ ]:
netflix.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7457 entries, 0 to 7456
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   movieId             0 non-null      float64
 1   title               7457 non-null   object 
 2   userId              0 non-null      float64
 3   rating              7457 non-null   float64
 4   timestamp           0 non-null      float64
 5   year                0 non-null      float64
 6   source              7457 non-null   object 
 7   type                7457 non-null   object 
 8   show_id             7457 non-null   object 
 9   director            7457 non-null   object 
 10  cast                7457 non-null   object 
 11  country             7457 non-null   object 
 12  date_added          7457 non-null   object 
 13  release_year        7457 non-null   float64
 14  duration            7457 non-null   object 
 15  description         7457 non-null   object 
 16  Genre 

In [ ]:
netflix_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7457 entries, 0 to 7456
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       7457 non-null   object
 1   type          7457 non-null   object
 2   title         7457 non-null   object
 3   director      7457 non-null   object
 4   cast          7457 non-null   object
 5   country       7457 non-null   object
 6   date_added    7457 non-null   object
 7   release_year  7457 non-null   int64 
 8   rating        7457 non-null   object
 9   duration      7457 non-null   object
 10  listed_in     7457 non-null   object
 11  description   7457 non-null   object
 12  Genre         7457 non-null   object
dtypes: int64(1), object(12)
memory usage: 757.5+ KB


In [ ]:
music.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27091 entries, 0 to 27090
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   movieId             0 non-null      float64
 1   title               0 non-null      float64
 2   userId              0 non-null      float64
 3   rating              27091 non-null  float64
 4   timestamp           0 non-null      float64
 5   year                0 non-null      float64
 6   source              27091 non-null  object 
 7   type                27091 non-null  object 
 8   show_id             0 non-null      float64
 9   director            0 non-null      float64
 10  cast                0 non-null      float64
 11  country             0 non-null      float64
 12  date_added          0 non-null      float64
 13  release_year        0 non-null      float64
 14  duration            0 non-null      float64
 15  description         0 non-null      float64
 16  Genr

In [ ]:
music_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56192 entries, 0 to 56191
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  56192 non-null  int64 
 1   Username    56192 non-null  object
 2   Artist      56192 non-null  object
 3   Track       56192 non-null  object
 4   Album       56192 non-null  object
 5   Date        56192 non-null  object
 6   Time        56192 non-null  object
 7   Genre       56192 non-null  object
dtypes: int64(1), object(7)
memory usage: 3.4+ MB


In [ ]:
movies_df = pd.read_csv('/content/drive/MyDrive/TFM/Datasets/integracionv1.csv')
netflix_df = pd.read_csv('/content/drive/MyDrive/TFM/Datasets/netflix_integracionv3.csv')
music_df = pd.read_csv('/content/drive/MyDrive/TFM/Datasets/musica_integracionv2.csv')


In [ ]:
movies_df.head()

,movieId,title,genres,userId,rating,timestamp,imdbId,tmdbId,year
0,1,Toy Story,Action|Children,1,4.0,2000-07-30 18:45:03,114709,862.0,1995
1,1,Toy Story,Action|Children,5,4.0,1996-11-08 06:36:02,114709,862.0,1995
2,1,Toy Story,Action|Children,7,4.5,2005-01-25 06:52:26,114709,862.0,1995
3,1,Toy Story,Action|Children,15,2.5,2017-11-13 12:59:30,114709,862.0,1995
4,1,Toy Story,Action|Children,17,4.5,2011-05-18 05:28:03,114709,862.0,1995


In [ ]:
import pandas as pd

# Cargar el dataset
movies_df = pd.read_csv('/content/drive/MyDrive/TFM/Datasets/integracionv1.csv')

# Mostrar la información general del dataset
print(movies_df.info())

# Mostrar las primeras filas del dataset para ver el contenido de las columnas
print(movies_df.head())

# Revisar si hay valores nulos en las columnas
print(movies_df.isnull().sum())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90267 entries, 0 to 90266
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   movieId    90267 non-null  int64  
 1   title      90267 non-null  object 
 2   genres     90267 non-null  object 
 3   userId     90267 non-null  int64  
 4   rating     90267 non-null  float64
 5   timestamp  90267 non-null  object 
 6   imdbId     90267 non-null  int64  
 7   tmdbId     90267 non-null  float64
 8   year       90267 non-null  int64  
dtypes: float64(2), int64(4), object(3)
memory usage: 6.2+ MB
None
   movieId      title           genres  userId  rating            timestamp  \
0        1  Toy Story  Action|Children       1     4.0  2000-07-30 18:45:03   
1        1  Toy Story  Action|Children       5     4.0  1996-11-08 06:36:02   
2        1  Toy Story  Action|Children       7     4.5  2005-01-25 06:52:26   
3        1  Toy Story  Action|Children      15     2.5  2017-11-13 12

In [ ]:
movies_df = pd.read_csv('/content/drive/MyDrive/TFM/Datasets/integracionv1.csv')

In [ ]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90267 entries, 0 to 90266
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   movieId    90267 non-null  int64  
 1   title      90267 non-null  object 
 2   genres     90267 non-null  object 
 3   userId     90267 non-null  int64  
 4   rating     90267 non-null  float64
 5   timestamp  90267 non-null  object 
 6   imdbId     90267 non-null  int64  
 7   tmdbId     90267 non-null  float64
 8   year       90267 non-null  int64  
dtypes: float64(2), int64(4), object(3)
memory usage: 6.2+ MB


In [ ]:
import pandas as pd

# Cargar el dataset
file_path = '/content/drive/MyDrive/TFM/Datasets/integracionv1.csv'
movies_df = pd.read_csv(file_path)

# Convertir 'genres' en listas y luego a cadenas directamente
movies_df['genres'] = movies_df['genres'].apply(lambda x: ', '.join(x.split('|')))

# Añadir columna 'emotion' vacía para etiquetar emociones
movies_df['emotion'] = None

# Eliminar las columnas 'imdbId' y 'tmdbId'
movies_df.drop(['imdbId', 'tmdbId'], axis=1, inplace=True)

# Crear la columna `bert_input` combinando atributos relevantes
movies_df['bert_input'] = (
    movies_df['title'] + " | " +
    movies_df['genres'] + " | " +
    movies_df['year'].astype(str) + " | " +
    "Rating: " + movies_df['rating'].astype(str)
)

# Mostrar los primeros registros para verificar los cambios
print(movies_df.head())

# Guardar el dataset preparado
output_path = '/content/drive/MyDrive/TFM/Datasets/movies_prepared_for_bert.csv'
movies_df.to_csv(output_path, index=False)

print(f"Archivo actualizado y guardado en: {output_path}")


   movieId      title            genres  userId  rating            timestamp  \
0        1  Toy Story  Action, Children       1     4.0  2000-07-30 18:45:03   
1        1  Toy Story  Action, Children       5     4.0  1996-11-08 06:36:02   
2        1  Toy Story  Action, Children       7     4.5  2005-01-25 06:52:26   
3        1  Toy Story  Action, Children      15     2.5  2017-11-13 12:59:30   
4        1  Toy Story  Action, Children      17     4.5  2011-05-18 05:28:03   

   year emotion                                         bert_input  
0  1995    None  Toy Story | Action, Children | 1995 | Rating: 4.0  
1  1995    None  Toy Story | Action, Children | 1995 | Rating: 4.0  
2  1995    None  Toy Story | Action, Children | 1995 | Rating: 4.5  
3  1995    None  Toy Story | Action, Children | 1995 | Rating: 2.5  
4  1995    None  Toy Story | Action, Children | 1995 | Rating: 4.5  
Archivo actualizado y guardado en: /content/drive/MyDrive/TFM/Datasets/movies_prepared_for_bert.csv


In [ ]:
# Cargar el dataset
movies_df = pd.read_csv('/content/drive/MyDrive/TFM/Datasets/movies_prepared_for_bert.csv')

# Mostrar la información general del dataset
print(movies_df.info())

# Mostrar las primeras filas del dataset para ver el contenido de las columnas
print(movies_df.head())

# Revisar si hay valores nulos en las columnas
print(movies_df.isnull().sum())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90267 entries, 0 to 90266
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   movieId     90267 non-null  int64  
 1   title       90267 non-null  object 
 2   genres      90267 non-null  object 
 3   userId      90267 non-null  int64  
 4   rating      90267 non-null  float64
 5   timestamp   90267 non-null  object 
 6   year        90267 non-null  int64  
 7   emotion     0 non-null      float64
 8   bert_input  90267 non-null  object 
dtypes: float64(2), int64(3), object(4)
memory usage: 6.2+ MB
None
   movieId      title            genres  userId  rating            timestamp  \
0        1  Toy Story  Action, Children       1     4.0  2000-07-30 18:45:03   
1        1  Toy Story  Action, Children       5     4.0  1996-11-08 06:36:02   
2        1  Toy Story  Action, Children       7     4.5  2005-01-25 06:52:26   
3        1  Toy Story  Action, Children      15     2.

In [ ]:
import pandas as pd


# Ruta al archivo
file_path = "/content/drive/MyDrive/TFM/Datasets/movies_prepared_for_bert.csv"

# Cargar el dataset
try:
    dataset = pd.read_csv(file_path)

    # Verificar duplicados basados en 'title', 'genres', y 'year'
    duplicados = dataset[dataset.duplicated(subset=['title', 'genres', 'year'], keep=False)]
    num_duplicados = len(duplicados)

    # Mostrar cuántos duplicados existen
    print(f"Registros duplicados encontrados: {num_duplicados}")

    # Guardar duplicados en un archivo separado para su análisis
    duplicados_file_path = "/content/drive/MyDrive/TFM/Datasets/movies_duplicates.csv"
    duplicados.to_csv(duplicados_file_path, index=False)
    print(f"Duplicados guardados en: {duplicados_file_path}")

    # Mostrar una muestra de los duplicados encontrados
    print(duplicados.head())

except FileNotFoundError:
    print(f"El archivo no se encuentra en la ruta: {file_path}. Verifica la ruta y vuelve a intentarlo.")



Registros duplicados encontrados: 90267
Duplicados guardados en: /content/drive/MyDrive/TFM/Datasets/movies_duplicates.csv
   movieId      title            genres  userId  rating            timestamp  \
0        1  Toy Story  Action, Children       1     4.0  2000-07-30 18:45:03   
1        1  Toy Story  Action, Children       5     4.0  1996-11-08 06:36:02   
2        1  Toy Story  Action, Children       7     4.5  2005-01-25 06:52:26   
3        1  Toy Story  Action, Children      15     2.5  2017-11-13 12:59:30   
4        1  Toy Story  Action, Children      17     4.5  2011-05-18 05:28:03   

   year  emotion                                         bert_input  
0  1995      NaN  Toy Story | Action, Children | 1995 | Rating: 4.0  
1  1995      NaN  Toy Story | Action, Children | 1995 | Rating: 4.0  
2  1995      NaN  Toy Story | Action, Children | 1995 | Rating: 4.5  
3  1995      NaN  Toy Story | Action, Children | 1995 | Rating: 2.5  
4  1995      NaN  Toy Story | Action, Children

In [ ]:
import pandas as pd

# Ruta del archivo original
file_path = "/content/drive/MyDrive/TFM/Datasets/movies_prepared_for_bert.csv"

# Cargar el dataset
dataset = pd.read_csv(file_path)

# Eliminar duplicados basados en 'title', 'genres', y 'year'
dataset_unique = dataset.drop_duplicates(subset=['title', 'genres', 'year'])

# Tomar una muestra única de 1000 registros
sample_size = 1000
sample_dataset = dataset_unique.sample(n=sample_size, random_state=42)

# Guardar la muestra en un archivo separado
sample_file_path = "/content/drive/MyDrive/TFM/Datasets/movies_sample_for_bert.csv"
sample_dataset.to_csv(sample_file_path, index=False)

print(f"Muestra única de {sample_size} registros guardada en: {sample_file_path}")



Muestra única de 1000 registros guardada en: /content/drive/MyDrive/TFM/Datasets/movies_sample_for_bert.csv


In [ ]:
sample_movies_df = pd.read_csv('/content/drive/MyDrive/TFM/Datasets/movies_sample_for_bert.csv')


In [ ]:
sample_movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   movieId     1000 non-null   int64  
 1   title       1000 non-null   object 
 2   genres      1000 non-null   object 
 3   userId      1000 non-null   int64  
 4   rating      1000 non-null   float64
 5   timestamp   1000 non-null   object 
 6   year        1000 non-null   int64  
 7   emotion     0 non-null      float64
 8   bert_input  1000 non-null   object 
dtypes: float64(2), int64(3), object(4)
memory usage: 70.4+ KB


In [ ]:
sample_movies_df.head()

,movieId,title,genres,userId,rating,timestamp,year,emotion,bert_input
0,2804,"Christmas Story, A","Children, Comedy",19,3.0,2000-08-08 03:32:52,1983,NaN,"Christmas Story, A | Children, Comedy | 1983 |..."
1,152081,Zootopia,"Action, Family",15,3.0,2017-11-13 13:00:39,2016,NaN,"Zootopia | Action, Family | 2016 | Rating: 3.0"
2,183897,Isle of Dogs,"Comedy, Family",212,5.0,2018-07-23 16:00:17,2018,NaN,"Isle of Dogs | Comedy, Family | 2018 | Rating:..."
3,80693,It's Kind of a Funny Story,"Comedy, Drama",62,4.0,2018-03-27 22:42:36,2010,NaN,"It's Kind of a Funny Story | Comedy, Drama | 2..."
4,68157,Inglourious Basterds,"Action, Drama",2,4.5,2015-10-24 19:32:34,2009,NaN,"Inglourious Basterds | Action, Drama | 2009 | ..."


In [ ]:
import pandas as pd

# Cargar el dataset
file_path = "/content/drive/MyDrive/TFM/Datasets/movies_sample_for_bert.csv"  # Cambia a la ruta correcta de tu archivo
dataset = pd.read_csv(file_path)

# Convertir la columna 'emotion' a texto para las etiquetas
dataset['emotion'] = dataset['emotion'].astype(str)

# Lista de emociones
emotion_options = [
    "Happy", "Sad", "Nostalgic", "Excited", "Scared",
    "Angry", "Romantic", "Hopeful", "Funny", "Adventurous",
    "Suspenseful", "Inspiring", "Heartwarming"
]

# Función para etiquetar manualmente con etiquetas compuestas
def etiquetar_manual(df):
    print("Inicio del proceso de etiquetado manual. Escribe '0' para guardar y salir.\n")
    for index, row in df.iterrows():
        if row['emotion'] == "nan" or row['emotion'] == "":  # Solo procesar los no etiquetados
            print(f"\n=== Registro {index + 1}/{len(df)} ===")
            print(f"Título: {row['title']}")
            print(f"Géneros: {row['genres']}")
            print(f"Año: {row['year']}")
            print(f"Rating: {row['rating']}")
            print(f"User ID: {row['userId']}")

            print("\nSelecciona una o más emociones separadas por comas (ej: 9,13):")
            for i, emotion in enumerate(emotion_options, 1):
                print(f"{i}. {emotion}")

            emotion_indices = input("\nEscribe los números de las emociones (o '0' para guardar y salir): ").strip()
            if emotion_indices == '0':
                print("\nGuardando progreso...")
                df.to_csv(file_path, index=False)
                print("Progreso guardado. ¡Hasta luego!")
                break

            try:
                selected_emotions = [emotion_options[int(i) - 1] for i in emotion_indices.split(",") if int(i) > 0 and int(i) <= len(emotion_options)]
                df.at[index, 'emotion'] = ", ".join(selected_emotions)
                print(f"Etiqueta '{', '.join(selected_emotions)}' guardada para el registro {index + 1}.\n")
            except (ValueError, IndexError):
                print("Entrada no válida. Intenta nuevamente con números separados por comas.")

    print("\n¡Todos los registros han sido etiquetados!")
    df.to_csv(file_path, index=False)
    return df

# Llamar a la función para iniciar el etiquetado
dataset = etiquetar_manual(dataset)



Inicio del proceso de etiquetado manual. Escribe '0' para guardar y salir.


=== Registro 1/1000 ===
Título: Christmas Story, A
Géneros: Children, Comedy
Año: 1983
Rating: 3.0
User ID: 19

Selecciona una o más emociones separadas por comas (ej: 9,13):
1. Happy
2. Sad
3. Nostalgic
4. Excited
5. Scared
6. Angry
7. Romantic
8. Hopeful
9. Funny
10. Adventurous
11. Suspenseful
12. Inspiring
13. Heartwarming

Escribe los números de las emociones (o '0' para guardar y salir): 1,3
Etiqueta 'Happy, Nostalgic' guardada para el registro 1.


=== Registro 2/1000 ===
Título: Zootopia
Géneros: Action, Family
Año: 2016
Rating: 3.0
User ID: 15

Selecciona una o más emociones separadas por comas (ej: 9,13):
1. Happy
2. Sad
3. Nostalgic
4. Excited
5. Scared
6. Angry
7. Romantic
8. Hopeful
9. Funny
10. Adventurous
11. Suspenseful
12. Inspiring
13. Heartwarming

Escribe los números de las emociones (o '0' para guardar y salir): 9,12
Etiqueta 'Funny, Inspiring' guardada para el registro 2.


=== Registro 3

In [ ]:
!pip install xlsxwriter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 2.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd

# Cargar el dataset de películas
file_path = "/content/drive/MyDrive/TFM/Datasets/movies_sample_for_bert.csv"
movies_dataset = pd.read_csv(file_path)

# Crear un archivo Excel para etiquetado manual
output_path = "/content/drive/MyDrive/TFM/Datasets/movies_sample_for_manual_labeling.xlsx"

# Lista de emociones para agregar como referencia
emotion_options = [
    "1. Happy", "2. Sad", "3. Nostalgic", "4. Excited", "5. Scared",
    "6. Angry", "7. Romantic", "8. Hopeful", "9. Funny",
    "10. Adventurous", "11. Suspenseful", "12. Inspiring", "13. Heartwarming"
]

# Crear un DataFrame para la lista de emociones
emotions_df = pd.DataFrame({"Emotion Options": emotion_options})

# Crear un escritor de Excel
with pd.ExcelWriter(output_path, engine='xlsxwriter') as writer:
    # Guardar el dataset en la primera hoja
    movies_dataset.to_excel(writer, sheet_name="Movies Dataset", index=False)
    # Guardar las emociones en una segunda hoja
    emotions_df.to_excel(writer, sheet_name="Emotion Options", index=False)

print(f"Archivo Excel para etiquetado guardado en: {output_path}")


Archivo Excel para etiquetado guardado en: /content/drive/MyDrive/TFM/Datasets/movies_sample_for_manual_labeling.xlsx


In [ ]:
import pandas as pd

# Ruta del archivo original con etiquetas
file_path = "/content/drive/MyDrive/TFM/Datasets/movies_sample_for_manual_labeling.xlsx"

# Cargar los datos
data = pd.read_excel(file_path)

# Mapeo de números a nombres de emociones
emotion_mapping = {
    1: "Happy", 2: "Sad", 3: "Nostalgic", 4: "Excited", 5: "Scared",
    6: "Angry", 7: "Romantic", 8: "Hopeful", 9: "Funny",
    10: "Adventurous", 11: "Suspenseful", 12: "Inspiring", 13: "Heartwarming"
}

# Función para convertir números a nombres y preservar el orden original
def map_emotions_preserve_order(emotion):
    if isinstance(emotion, str):
        # Dividir y mapear números a nombres respetando el orden
        parts = [part.strip() for part in emotion.split(',')]
        return ", ".join([emotion_mapping[int(part)] if part.isdigit() else part for part in parts])
    return emotion

# Aplicar la transformación a la columna 'emotion'
data['emotion'] = data['emotion'].apply(map_emotions_preserve_order)

# Guardar el archivo procesado
output_path = "/content/drive/MyDrive/TFM/Datasets/movies_sample_ready_for_bert.csv"
data.to_csv(output_path, index=False)

print(f"Archivo con emociones homologadas y orden preservado guardado en: {output_path}")


Archivo con emociones homologadas y orden preservado guardado en: /content/drive/MyDrive/TFM/Datasets/movies_sample_ready_for_bert.csv


# Sistema de Recomendación

**Archivos Finales para trabajar con BERT**

**Muestra**

In [ ]:
sample_movies_ready = pd.read_csv('/content/drive/MyDrive/TFM/Datasets/movies_sample_ready_for_bert.csv')

In [ ]:
sample_movies_ready.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   movieId      1000 non-null   int64  
 1   title        1000 non-null   object 
 2   genres       1000 non-null   object 
 3   userId       1000 non-null   int64  
 4   rating       1000 non-null   float64
 5   timestamp    1000 non-null   object 
 6   year         1000 non-null   int64  
 7   bert_input   1000 non-null   object 
 8   emotion      1000 non-null   object 
 9   Description  1000 non-null   object 
dtypes: float64(1), int64(3), object(6)
memory usage: 78.2+ KB


In [ ]:
sample_movies_ready.head()

,movieId,title,genres,userId,rating,timestamp,year,bert_input,emotion,Description
0,2804,"Christmas Story, A","Children, Comedy",19,3.0,2000-08-08 03:32:52,1983,"Christmas Story, A | Children, Comedy | 1983 |...","Happy, Nostalgic","Set in the 1940s, young Ralphie Parker dreams ..."
1,152081,Zootopia,"Action, Family",15,3.0,2017-11-13 13:00:39,2016,"Zootopia | Action, Family | 2016 | Rating: 3.0","Adventurous, Inspiring","In a city where animals coexist, a bunny polic..."
2,183897,Isle of Dogs,"Comedy, Family",212,5.0,2018-07-23 16:00:17,2018,"Isle of Dogs | Comedy, Family | 2018 | Rating:...","Inspiring, Happy",In a dystopian future where dogs are exiled to...
3,80693,It's Kind of a Funny Story,"Comedy, Drama",62,4.0,2018-03-27 22:42:36,2010,"It's Kind of a Funny Story | Comedy, Drama | 2...","Inspiring, Sad",A teenager checks himself into a mental health...
4,68157,Inglourious Basterds,"Action, Drama",2,4.5,2015-10-24 19:32:34,2009,"Inglourious Basterds | Action, Drama | 2009 | ...","Suspenseful, Angry",A group of Jewish American soldiers plan to as...


In [ ]:
sample_movies_ready.emotion.value_counts()

,count
emotion,
"Inspiring, Sad",91
"Funny, Happy",82
"Scared, Suspenseful",61
"Happy, Funny",58
"Adventurous, Suspenseful",48
...,...
"Heartwarming, Adventurous",1
"Excited, Nostalgic",1
"Suspenseful, Hopeful",1


In [ ]:
sample_movies_ready.emotion.unique()

array(['Happy, Nostalgic', 'Adventurous, Inspiring', 'Inspiring, Happy',
       'Inspiring, Sad', 'Suspenseful, Angry', 'Happy, Funny',
       'Suspenseful, Romantic', 'Sad, Suspenseful', 'Sad, Inspiring',
       'Adventurous, Happy', 'Scared, Suspenseful', 'Romantic, Inspiring',
       'Adventurous, Romantic', 'Happy, Adventurous',
       'Adventurous, Funny', 'Angry, Sad', 'Funny, Adventurous',
       'Romantic, Sad', 'Suspenseful, Sad', 'Romantic, Happy',
       'Adventurous, Suspenseful', 'Adventurous, Angry', 'Sad, Romantic',
       'Scared, Inspiring', 'Romantic, Funny', 'Suspenseful, Scared',
       'Angry, Suspenseful', 'Funny, Suspenseful',
       'Suspenseful, Adventurous', 'Suspenseful, Hopeful', 'Funny, Happy',
       'Excited, Nostalgic', 'Romantic, Adventurous',
       'Adventurous, Heartwarming', 'Sad, Heartwarming', 'Sad, Angry',
       'Happy, Romantic', 'Heartwarming, Sad', 'Romantic, Suspenseful',
       'Inspiring, Adventurous', 'Funny, Romantic', 'Angry, Inspiring'

In [ ]:
sample_movies_ready.to_csv('/content/drive/MyDrive/TFM/Dataset_for_bert/sample_movies_ready_for_bert.csv', index=False)

**Archivo Grande**

In [ ]:

movies_ready = pd.read_csv('/content/drive/MyDrive/TFM/Datasets/movies_prepared_for_bert.csv')

In [ ]:
movies_ready.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90267 entries, 0 to 90266
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   movieId     90267 non-null  int64  
 1   title       90267 non-null  object 
 2   genres      90267 non-null  object 
 3   userId      90267 non-null  int64  
 4   rating      90267 non-null  float64
 5   timestamp   90267 non-null  object 
 6   year        90267 non-null  int64  
 7   emotion     0 non-null      float64
 8   bert_input  90267 non-null  object 
dtypes: float64(2), int64(3), object(4)
memory usage: 6.2+ MB


In [ ]:
movies_ready.head()

,movieId,title,genres,userId,rating,timestamp,year,emotion,bert_input
0,1,Toy Story,"Action, Children",1,4.0,2000-07-30 18:45:03,1995,NaN,"Toy Story | Action, Children | 1995 | Rating: 4.0"
1,1,Toy Story,"Action, Children",5,4.0,1996-11-08 06:36:02,1995,NaN,"Toy Story | Action, Children | 1995 | Rating: 4.0"
2,1,Toy Story,"Action, Children",7,4.5,2005-01-25 06:52:26,1995,NaN,"Toy Story | Action, Children | 1995 | Rating: 4.5"
3,1,Toy Story,"Action, Children",15,2.5,2017-11-13 12:59:30,1995,NaN,"Toy Story | Action, Children | 1995 | Rating: 2.5"
4,1,Toy Story,"Action, Children",17,4.5,2011-05-18 05:28:03,1995,NaN,"Toy Story | Action, Children | 1995 | Rating: 4.5"


In [ ]:
movies_ready.to_csv('/content/drive/MyDrive/TFM/Dataset_for_bert/movies_ready_for_bert.csv', index=False)